# Bloco 1: Importação de Bibliotecas e Configurações Iniciais
**Descrição:** Importa as bibliotecas necessárias para a análise (pandas, glob, numpy, matplotlib) e define configurações de exibição para o pandas, garantindo que mais colunas e linhas sejam mostradas nas saídas.

In [2]:
#| classes: [fold-output]  # ( saída recolhível)
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

# Config exibição do Pandas
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

# Bloco 2: Carregamento e Leitura dos Dados de Estoque
**Descrição:** Localiza todos os arquivos CSV de estoque, define os tipos de dados para colunas e carrega e concatena todos os arquivos em um único DataFrame.

 <span style="color:red;"> Aviso importante, abra pra ler</span>

## Paths
**Atenção:** <span style="color:orange;">Redefina o Caminho para os arquivos</span>

In [3]:
caminho_feriados = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\feriados_nacionais.xls'
patt = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\data_temp\fct_2025-07-14\*Estoque*.csv'  

## colunas

In [4]:
#| classes: [fold-output]  # ( saída recolhível)
list_files = glob(patt)
print("Arquivos encontrados:", list_files)

colunas_data = [
    'DataEmissao', 'DataAquisicao', 'DataVencimento', 'DataGeracao'
]

colunas_texto = [
    'Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf', 'TIT_CEDENTE_ENT_CODIGO',
    'CedenteNome', 'Cnae', 'SecaoCNAEDescricao', 'NotaPdd', 'SAC_TIPO_PESSOA',
    'SacadoCnpjCpf', 'SacadoNome', 'IdTituloVortx', 'TipoAtivo', 'NumeroBoleto',
    'NumeroTitulo', 'CampoChave', 'PagamentoParcial', 'Coobricacao',
    'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3', 'CampoAdicional4',
    'CampoAdicional5', 'IdTituloVortxOriginador', 'Registradora',
    'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio'
]


dtype_texto = {col: str for col in colunas_texto}
"""dic de tips para colns de texto"""



Arquivos encontrados: ['C:\\Users\\Leo\\Desktop\\Porto_Real\\portoauto\\data_temp\\fct_2025-07-14\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte1.csv', 'C:\\Users\\Leo\\Desktop\\Porto_Real\\portoauto\\data_temp\\fct_2025-07-14\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte2.csv', 'C:\\Users\\Leo\\Desktop\\Porto_Real\\portoauto\\data_temp\\fct_2025-07-14\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte3.csv', 'C:\\Users\\Leo\\Desktop\\Porto_Real\\portoauto\\data_temp\\fct_2025-07-14\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte4.csv']


'dic de tips para colns de texto'

## Leitura e Concatenacao

In [5]:
#| classes: [fold-output]  # ( saída recolhível)
dfs = []
for file in list_files:
    print(f"Lendo o arquivo: {file}")
    df_ = pd.read_csv(file, sep=';', encoding='latin1', dtype=dtype_texto,
                      decimal=',', parse_dates=colunas_data, dayfirst=True)
    dfs.append(df_)

df_final = pd.concat(dfs, ignore_index=True)
print("Leitura e concatenação concluídas.")

Lendo o arquivo: C:\Users\Leo\Desktop\Porto_Real\portoauto\data_temp\fct_2025-07-14\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte1.csv
Lendo o arquivo: C:\Users\Leo\Desktop\Porto_Real\portoauto\data_temp\fct_2025-07-14\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte2.csv
Lendo o arquivo: C:\Users\Leo\Desktop\Porto_Real\portoauto\data_temp\fct_2025-07-14\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte3.csv
Lendo o arquivo: C:\Users\Leo\Desktop\Porto_Real\portoauto\data_temp\fct_2025-07-14\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte4.csv
Leitura e concatenação concluídas.


# Bloco 3: Criação de Colunas Auxiliares
(para facilitar análises)
- `_ValorLiquido`: Valor Presente ajustado pelo PDD. (-PDDTOTAL)
- `_ValorVencido`: Identifica o valor de títulos já vencidos. ()
- `_MuitosContratos`: Flag para sacados com um número elevado de contratos (CCB).
- `_MuitosEntes`: Flag para sacados associados a múltiplos convênios.

In [6]:
#| classes: [fold-output] 
# ( saída recolhível)
# Criar colunas aux
df_final['_ValorLiquido'] = df_final['ValorPresente'] - df_final['PDDTotal']

df_final['_ValorVencido'] = (df_final['DataVencimento'] <= df_final['DataGeracao']).astype('int') * df_final['ValorPresente']

# * sacados com muitos contratos (CCB)
sacado_contratos = df_final.groupby('SacadoNome')['CCB'].nunique()
k = 3
mask_contratos = sacado_contratos >= k
sacado_contratos_alto = sacado_contratos[mask_contratos].index
df_final['_MuitosContratos'] = df_final['SacadoNome'].isin(sacado_contratos_alto).astype(str)

#*  sacados com muitos entes (Convênios)
sacados_entes = df_final.groupby('SacadoCnpjCpf')['Convênio'].nunique()
k2 = 3
mask_entes = sacados_entes >= k2
sacados_entes_alto = sacados_entes[mask_entes].index
df_final['_MuitosEntes'] = df_final['SacadoCnpjCpf'].isin(sacados_entes_alto).astype(str)

print("Criação de colunas auxiliares concluída.")

Criação de colunas auxiliares concluída.


# Bloco 4: Verificação Inicial do DataFrame
**Descrição:** Realiza uma verificação rápida do DataFrame, mostrando o uso de memória e o valor total do estoque (Valor Presente).

In [7]:
#| classes: [fold-output] 
# ( saída recolhível)
memoria_mb = df_final.memory_usage(deep=True).sum() / 1024**2
print(f"Uso de memória do DataFrame: {memoria_mb:.2f} MB")

valor_total_estoque = df_final["ValorPresente"].sum()
print(f"Valor Presente Total do Estoque: R$ {valor_total_estoque:_.2f}".replace('.', ',').replace('_', '.'))

Uso de memória do DataFrame: 2733.50 MB
Valor Presente Total do Estoque: R$ 199.232.147,56


# Bloco 5: Análise Exploratória Geral Contagens
**Descrição:** Itera sobre as colunas de texto para entender a distribuição de suas categorias. As perguntas no comentário guiam esta exploração.

<span style="color:yellow;"> **Questões para investigação:** </span>
1.  `[Situacao]` - diferença entre sem cobrança e aditado
2.  `[SAC_TIPO_PESSOA]` - tipo J = jurídico?? tem isso?
3.  `[SacadoCnpjCpf]` - por que tem CNPJ?
4.  `[SacadoNome]` 'BMP SOCIEDADE DE CREDITO DIRETO S.A'
5.  `[SacadoCnpjCpf']` - verificar consistência dos CPFs
6.  `[TipoAtivo]` - CCB e Contrato. Por que tem contrato?
7.  `[DataGeracao]` - data de referência ou data de processamento?
8.  `[SacadoCnpjCpf]` - tem sacado com 1040 linhas (!)
9.  `[Convênio]` - sacados com muitos convênios (3)

In [8]:
#| classes: [fold-output] 
# ( saída recolhível)
df_final2 = df_final[~df_final['Situacao'].isna()].copy()

print("Analisando a contagem de valores para colunas de texto (geral):")
for col in df_final2.select_dtypes(include=['object']).columns:
    print(f"--- Análise da coluna: {col} ---")
    print(df_final2[col].value_counts(dropna=False))
    print('*' * 80)

Analisando a contagem de valores para colunas de texto (geral):
--- Análise da coluna: Situacao ---
Situacao
Sem cobranÃ§a    1471376
Aditado            50939
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: PES_TIPO_PESSOA ---
PES_TIPO_PESSOA
J    1522315
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: CedenteCnpjCpf ---
CedenteCnpjCpf
34.337.707/0001-00    1519907
92.874.270/0001-40       2408
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: TIT_CEDENTE_ENT_CODIGO ---
TIT_CEDENTE_ENT_CODIGO
318853     1519907
3224693       2408
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: CedenteNome ---
CedenteNome
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A    1519907
BANC

# Bloco 6: Verificação do Impacto de Linhas com 'Situacao' Nula
**Descrição:** Analisa se as linhas com 'Situacao' nula têm impacto significativo no cálculo do Valor Presente total.

In [9]:
#| classes: [fold-output] 
# ( saída recolhível)
print("Verificando o impacto das linhas com 'Situacao' nula...")
v1_ = df_final['ValorPresente'].sum()
v2_ = df_final[~df_final['Situacao'].isna()]['ValorPresente'].sum()
print(f"Valores são próximos? {np.isclose(v1_, v2_)}")

# Libera memória
del df_final

Verificando o impacto das linhas com 'Situacao' nula...
Valores são próximos? True


# Bloco 7: Análise de Sacados com SAC_TIPO_PESSOA == 'J'
**Descrição:** Isola e analisa registros onde o tipo de sacado é 'J'.

In [10]:
#| classes: [fold-output] 
# ( saída recolhível)
print("Analisando sacados com tipo 'J'...")
df_sacado_J = df_final2[df_final2['SAC_TIPO_PESSOA'] == 'J']
if not df_sacado_J.empty:
    print("Tamanhos de 'SacadoCnpjCpf' para tipo 'J':", df_sacado_J['SacadoCnpjCpf'].map(len).unique())
    display(df_sacado_J.sample(min(5, len(df_sacado_J))))
else:
    print("Nenhum sacado do tipo 'J' encontrado.")

Analisando sacados com tipo 'J'...
Tamanhos de 'SacadoCnpjCpf' para tipo 'J': [14 18]


,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes
1443754,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,009.598.794-04,JANAINA TELMA DA SILVA GOMES,91801513,CCB,2025-05-06,2025-05-09,2030-07-20,NaN,54296536062,54296536062,9.09,43.33,9.5949,0.0,0.0,NAO,NAO,2025-07-14,0.0000,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0,NaN,NaN,NaN,54296536,GOV. RIO GRANDE DO NORTE,9.5949,0.0,False,False
690384,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,284.380.118-45,DELCILENE ROCHA DA SILVA,67932139,CCB,2024-12-03,2024-12-04,2029-12-20,NaN,45056011060,182337926,8.92,45.81,10.8365,0.0,0.0,NAO,NAO,2025-07-14,7.7970,NaN,NaN,NaN,NaN,NaN,7.796994,0.0,42657392,NaN,NaN,NaN,45056011,PREF. TUTOIA,3.0395,0.0,True,False
979972,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,429.812.474-68,FRANCISCO CANINDE DE MIRANDA,48542500,CCB,2024-07-25,2024-07-25,2028-04-27,NaN,3841410545,179841719,63.53,178.08,82.7898,0.0,0.0,NAO,NAO,2025-07-14,0.0000,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0,NaN,NaN,NaN,38414105,GOV. RIO GRANDE DO NORTE,82.7898,0.0,False,False
613067,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,045.478.369-81,ANA MARIA DENARDES DO AMARAL GONCALVES,38046379,CCB,2024-04-23,2024-04-24,2028-03-15,NaN,3484358546,80044578,37.68,110.00,52.7034,0.0,0.0,NAO,NAO,2025-07-14,52.7034,NaN,NaN,NaN,NaN,NaN,52.703400,1.0,38045466,NaN,NaN,NaN,34843585,PREF. MARINGA,0.0000,0.0,False,False
142545,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,167.207.761-34,ESTHER BARBOSA DE SOUSA,40750804,CCB,2024-05-20,2024-05-23,2025-09-16,NaN,3590962215,143672245,402.65,569.34,542.6590,0.0,0.0,NAO,NAO,2025-07-14,0.0000,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0,NaN,NaN,NaN,35909622,GOV. GOIAS,542.6590,0.0,False,False


# Bloco 8: Análise Específica de Sacados com CNPJ
**Descrição:** Filtra sacados com CNPJ e realiza uma análise detalhada sobre eles, incluindo uma nova verificação de value_counts() para todas as colunas categóricas dentro deste subconjunto.

In [11]:
#| classes: [fold-output] 
# ( saída recolhível)
print("Analisando sacados com CNPJ...")
df_final2_cnpj = df_final2[df_final2['SacadoCnpjCpf'].map(len) == 18].copy()

if not df_final2_cnpj.empty:
    print("\nNomes únicos de sacados com CNPJ:")
    print(df_final2_cnpj['SacadoNome'].unique())

    print("\n--- value_counts() para o subconjunto de Sacados com CNPJ ---")
    for col in df_final2_cnpj.select_dtypes(include=['object']).columns:
        print(f"--- Análise da coluna: {col} (Apenas CNPJ) ---")
        print(df_final2_cnpj[col].value_counts(dropna=False))
        print('*'*80)

    print("\nEstatísticas descritivas para dados numéricos de sacados com CNPJ:")
    display(df_final2_cnpj.describe(include=[np.number]))

    print("\nSoma dos valores numéricos:")
    display(df_final2_cnpj.select_dtypes(include=[np.number]).sum())

    # Exportar para Excel
    # df_final2_cnpj.to_excel('df_final2_cnpj.xlsx')
else:
    print("Nenhum sacado com formato de CNPJ (18 caracteres) encontrado.")

Analisando sacados com CNPJ...

Nomes únicos de sacados com CNPJ:
['BMP SOCIEDADE DE CREDITO DIRETO S.A']

--- value_counts() para o subconjunto de Sacados com CNPJ ---
--- Análise da coluna: Situacao (Apenas CNPJ) ---
Situacao
Sem cobranÃ§a    95
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: PES_TIPO_PESSOA (Apenas CNPJ) ---
PES_TIPO_PESSOA
J    95
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: CedenteCnpjCpf (Apenas CNPJ) ---
CedenteCnpjCpf
34.337.707/0001-00    95
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: TIT_CEDENTE_ENT_CODIGO (Apenas CNPJ) ---
TIT_CEDENTE_ENT_CODIGO
318853    95
Name: count, dtype: int64
********************************************************************************
--- Análise da coluna: CedenteNome (Apenas CNPJ

,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PDDTotal,PDDEfeitoVagao,PercentagemEfeitoVagao,_ValorLiquido,_ValorVencido
count,95.000000,9.500000e+01,95.000000,95.0,95.000000,95.000000,95.000000,95.0,95.0,95.000000
mean,23.177368,5.613000e+01,27.604434,0.0,1.650542,27.604434,27.604434,1.0,0.0,4.135895
std,13.054230,1.428624e-14,15.121244,0.0,8.263526,15.121244,15.121244,0.0,0.0,14.742111
min,7.310000,5.613000e+01,8.795000,0.0,0.000000,8.795000,8.795000,1.0,0.0,0.000000
25%,12.005000,5.613000e+01,14.445900,0.0,0.000000,14.445900,14.445900,1.0,0.0,0.000000
50%,19.710000,5.613000e+01,23.720600,0.0,0.000000,23.720600,23.720600,1.0,0.0,0.000000
75%,32.350000,5.613000e+01,38.939150,0.0,0.000000,38.939150,38.939150,1.0,0.0,0.000000
max,53.070000,5.613000e+01,56.130000,0.0,56.130000,56.130000,56.130000,1.0,0.0,56.130000



Soma dos valores numéricos:


ValorAquisicao            2201.8500
ValorNominal              5332.3500
ValorPresente             2622.4212
PDDNota                      0.0000
PDDVencido                 156.8015
PDDTotal                  2622.4212
PDDEfeitoVagao            2622.4212
PercentagemEfeitoVagao      95.0000
_ValorLiquido                0.0000
_ValorVencido              392.9100
dtype: float64

# Bloco 9: Validação de CPFs na Coluna 'SacadoCnpjCpf'
**Descrição:** Aplica uma função para validar a estrutura matemática de CPFs.

In [12]:
#| classes: [fold-output] 
# ( saída recolhível)
def validar_cpf(cpf):
    """Função para validar um número de CPF."""
    #? note que ela nao ta dando fill à esquerda- o que nao importa tanto pq a string tá completa
    cpf = ''.join(filter(str.isdigit, str(cpf)))
    if len(cpf) != 11 or cpf == cpf[0] * 11: return False
    soma = sum(int(cpf[i]) * (10 - i) for i in range(9))
    resto = (soma * 10) % 11
    if resto == 10: resto = 0
    if resto != int(cpf[9]): return False
    soma = sum(int(cpf[i]) * (11 - i) for i in range(10))
    resto = (soma * 10) % 11
    if resto == 10: resto = 0
    if resto != int(cpf[10]): return False
    return True

mask_cpf = df_final2['SacadoCnpjCpf'].map(len) == 14
df_final2['CPF_válido'] = False
df_final2.loc[mask_cpf, 'CPF_válido'] = df_final2.loc[mask_cpf, 'SacadoCnpjCpf'].apply(validar_cpf)
cpfs_invalidos = df_final2[(mask_cpf) & (~df_final2['CPF_válido'])]['SacadoCnpjCpf'].unique()
print(f"Encontrados {len(cpfs_invalidos)} CPFs com estrutura inválida.")

Encontrados 0 CPFs com estrutura inválida.


# Bloco 10: Análise do Percentual de PDD por Variável Categórica
**Descrição:** Calcula o percentual de PDD agrupado por diversas variáveis categóricas para identificar categorias de maior risco.

In [13]:
#| classes: [fold-output] 
# ( saída recolhível)
cat_cols = [
    'Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial',
    'TipoAtivo', '_MuitosContratos', '_MuitosEntes', 'Convênio'
]

pdd_ref = (1 - df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"PDD de Referência (Total): {pdd_ref:.2f}%\n")

for col in cat_cols:
    print(f"--- Análise de PDD por '{col}' ---")
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum() / 1e6
    aux_['%PDD'] = (1 - aux_['_ValorLiquido'] / aux_['ValorPresente']) * 100
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
    display(aux_.head(20))
    print("\n" + "="*80 + "\n")

PDD de Referência (Total): 19.97%

--- Análise de PDD por 'Situacao' ---


,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
Aditado,7.851224,8.081707,2.851911
Sem cobranÃ§a,151.600857,191.150441,20.690292




--- Análise de PDD por 'CedenteNome' ---


,_ValorLiquido,ValorPresente,%PDD
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.246985,0.297263,16.913659
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,159.205096,198.934885,19.971253




--- Análise de PDD por 'SAC_TIPO_PESSOA' ---


...

,_ValorLiquido,ValorPresente,%PDD
_MuitosContratos,,,
False,137.633688,170.629483,19.337687
True,21.818393,28.602664,23.719020




--- Análise de PDD por '_MuitosEntes' ---


,_ValorLiquido,ValorPresente,%PDD
_MuitosEntes,,,
False,159.436245,199.208493,19.965137
True,0.015835,0.023654,33.054812




--- Análise de PDD por 'Convênio' ---


,_ValorLiquido,ValorPresente,%PDD
Convênio,,,
GOV. MARANHAO,41.484712,46.568145,10.916117
GOV. ALAGOAS,32.999110,34.786090,5.137055
GOV. GOIAS,13.734775,17.185891,20.081099
PREF. BARREIRAS,9.304598,9.363838,0.632651
PREF. CABO FRIO,2.354433,6.814405,65.449174
GOV. RIO GRANDE DO NORTE,6.043314,6.765837,10.678985
PREF. CAMPOS DOS GOYTACAZES,4.330700,6.313879,31.409832
PREF. BALSAS,5.092831,5.119877,0.528237
GOV. TOCANTINS,2.902283,5.105860,43.157807


# Bloco 11: Análise do Percentual de Títulos Vencidos por Variável Categórica
**Descrição:** Calcula a proporção do valor vencido em relação ao Valor Presente para cada categoria, identificando os grupos com maior inadimplência.

In [14]:
#| classes: [fold-output] 
# ( saída recolhível)
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"Percentual de Vencidos de Referência (Total): {venc_ref:.2f}%\n")

for col in cat_cols:
    print(f"--- Análise de Vencidos por '{col}' ---")
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum() / 1e6
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
    else:
        aux_ = aux_.sort_values('%Vencido', ascending=False)
    display(aux_.head(20))
    print("\n" + "="*80 + "\n")

Percentual de Vencidos de Referência (Total): 5.12%

--- Análise de Vencidos por 'Situacao' ---


,_ValorVencido,ValorPresente,%Vencido
Situacao,,,
Sem cobranÃ§a,10.165953,191.150441,5.318299
Aditado,0.038819,8.081707,0.480327




--- Análise de Vencidos por 'CedenteNome' ---


,_ValorVencido,ValorPresente,%Vencido
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.019911,0.297263,6.698064
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,10.184861,198.934885,5.119696




--- Análise de Vencidos por 'SAC_TIPO_PESSOA' ---


...

,_ValorVencido,ValorPresente,%Vencido
_MuitosContratos,,,
False,8.923531,170.629483,5.229771
True,1.281240,28.602664,4.479443




--- Análise de Vencidos por '_MuitosEntes' ---


,_ValorVencido,ValorPresente,%Vencido
_MuitosEntes,,,
True,0.001815,0.023654,7.672974
False,10.202956,199.208493,5.121748




--- Análise de Vencidos por 'Convênio' ---


,_ValorVencido,ValorPresente,%Vencido
Convênio,,,
GOV. MARANHAO,0.909802,46.568145,1.953700
GOV. ALAGOAS,1.368971,34.786090,3.935397
GOV. GOIAS,1.110039,17.185891,6.459012
PREF. BARREIRAS,0.018141,9.363838,0.193732
PREF. CABO FRIO,0.873612,6.814405,12.820073
GOV. RIO GRANDE DO NORTE,0.177908,6.765837,2.629507
PREF. CAMPOS DOS GOYTACAZES,0.449292,6.313879,7.115950
PREF. BALSAS,0.009430,5.119877,0.184182
GOV. TOCANTINS,0.499738,5.105860,9.787537


# Bloco 12: Verificação de Sacados Presentes em Múltiplos Convênios ('Entes')
**Descrição:** Identifica sacados com pulverização entre diferentes parceiros.

In [15]:
#| classes: [fold-output] 
# ( saída recolhível)

print("Analisando sacados presentes em múltiplos convênios...")
sacados_multi_entes = df_final2.groupby('SacadoCnpjCpf')['Convênio'].agg(['nunique', pd.unique])
sacados_multi_entes = sacados_multi_entes.sort_values('nunique', ascending=False)
display(sacados_multi_entes.head(35))

Analisando sacados presentes em múltiplos convênios...


,nunique,unique
SacadoCnpjCpf,,
048.692.204-90,3,"[GOV. GOIAS, PREF. TUTOIA, GOV. RIO GRANDE DO ..."
323.026.574-20,3,"[GOV. GOIAS, GOV. SÃO PAULO, GOV. RIO GRANDE D..."
292.798.363-15,2,"[GOV. MARANHAO, PREF. BALSAS]"
443.808.794-53,2,"[PREF. SÃO JOSÉ DE MIPIBU, GOV. RIO GRANDE DO ..."
813.733.763-68,2,"[GOV. MARANHAO, PREF. BALSAS]"
647.592.703-10,2,"[GOV. MARANHAO, PREF. SÃO LUIS]"
082.249.978-98,2,"[GOV. RIO GRANDE DO NORTE, PREF. SÃO JOSÉ DE M..."
054.906.964-00,2,"[GOV. RIO GRANDE DO NORTE, PREF. CAMPOS DOS GO..."
268.919.048-62,2,"[PREF. MAUÁ, PREF. SANTO ANDRE]"


# Bloco 13: Verificação de Consistência das Datas
**Descrição:** Realiza verificações do tipo *"sanity check"* (algo que deveria ser óbvio) nas colunas de data.

In [16]:
#| classes: [fold-output] 
# ( saída recolhível)

print("Verificando consistência das datas...")
check1 = (df_final2['DataEmissao'] > df_final2['DataAquisicao']).sum()
print(f"Registros com Data de Emissão > Data de Aquisição: {check1}")
check2 = (df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum()
print(f"Registros com Data de Aquisição > Data de Vencimento: {check2}")

Verificando consistência das datas...
Registros com Data de Emissão > Data de Aquisição: 0
Registros com Data de Aquisição > Data de Vencimento: 0


# Bloco 14: Verificação de Consistência dos Valores Monetários
**Descrição:** R"sanity check" nos valores financeiros.

In [17]:
#| classes: [fold-output] 
# ( saída recolhível)

print("\nVerificando consistência dos valores...")
check_v1 = (df_final2['ValorAquisicao'] > df_final2['ValorNominal']).sum()
print(f"Registros com Valor de Aquisição > Valor Nominal: {check_v1}")
check_v2 = (df_final2['ValorAquisicao'] > df_final2['ValorPresente']).sum()
print(f"Registros com Valor de Aquisição > Valor Presente: {check_v2}")
check_v3 = (df_final2['ValorPresente'] > df_final2['ValorNominal']).sum()
print(f"Registros com Valor Presente > Valor Nominal: {check_v3}")


Verificando consistência dos valores...
Registros com Valor de Aquisição > Valor Nominal: 3696
Registros com Valor de Aquisição > Valor Presente: 3696
Registros com Valor Presente > Valor Nominal: 0


# Bloco 15: Análise Focada de Métricas de Desempenho (PDD e Vencidos)
**Descrição:** Este bloco executa uma análise direcionada das métricas de PDD e Vencidos, segmentando os resultados por Cedente, Tipo de Contrato (TipoAtivo) e Ente Consignado (Convênio), conforme solicitado.


In [18]:
#| classes: [fold-output] 
# ( saída recolhível)
cat_cols = [
    'Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial',
    'TipoAtivo', '_MuitosContratos', '_MuitosEntes', 'Convênio'
]
# List ds dimensoes para a análise
dimensoes_analise = {
    'Cedentes': 'CedenteNome',
    'Tipo de Contrato': 'TipoAtivo',
    'Ente Consignado': 'Convênio', 
    'Situação': 'Situacao', 
    'Tipo de Pessoa Sacado':'SAC_TIPO_PESSOA', 
    'Pagamento Parcial': 'PagamentoParcial',
    'Tipo de Ativo':'TipoAtivo',
    'Tem Muitos Contratos':'_MuitosContratos', 
    'Tem Muitos Entes':'_MuitosEntes'
}

print("="*80)
print("INICIANDO ANÁLISE FOCADA DE MÉTRICAS DE DESEMPENHO")
print("="*80)

#* PERCENTUAL DE PDD POR SEGMENTO
print("\n\n--- [Análise de Risco: % PDD] ---\n")
pdd_ref = (1 - df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"PDD de Referência da Carteira Total: {pdd_ref:.2f}%\n")

for nome_analise, coluna in dimensoes_analise.items():
    print(f"--> Análise de PDD por '{nome_analise}' (Coluna: '{coluna}')")

    #  Agrupa os dados
    #  soma os valores 
    #  converte para milhões (para melhor leitura)
    aux_pdd = df_final2.groupby(coluna)[['_ValorLiquido', 'ValorPresente']].sum() / 1e6
    aux_pdd.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorLiquido': 'ValorLiquido (M)'}, inplace=True)

    # Calcula o %PDD
    aux_pdd['%PDD'] = (1 - aux_pdd['ValorLiquido (M)'] / aux_pdd['ValorPresente (M)']) * 100

    # maiores segmentos primeiro
    aux_pdd = aux_pdd.sort_values('ValorPresente (M)', ascending=False)

    # Exibe a tabela com os resultados
    display(aux_pdd.head(20))
    print("\n")


#* PERCENTUAL DE VENCIDOS POR SEGMENTO 
print("\n\n" + "="*80)
print("\n--- [Análise de Inadimplência: % Vencido] ---\n")
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"Percentual de Vencidos da Carteira Total: {venc_ref:.2f}%\n")

for nome_analise, coluna in dimensoes_analise.items():
    print(f"--> Análise de Vencidos por '{nome_analise}' (Coluna: '{coluna}')")

    # Agrupa os dados, soma os valores e converte para milhões
    aux_venc = df_final2.groupby(coluna)[['_ValorVencido', 'ValorPresente']].sum() / 1e6
    aux_venc.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorVencido': 'ValorVencido (M)'}, inplace=True)
    
    aux_venc['%Vencido'] = (aux_venc['ValorVencido (M)'] / aux_venc['ValorPresente (M)']) * 100

    #* Ordena p %Vencido
    aux_venc = aux_venc.sort_values('%Vencido', ascending=False)

    #  tabela    #! só no jupyter
    display(aux_venc.head(20))
    print("\n")

print("="*80)
print("FIM DA ANÁLISE")
print("="*80)

INICIANDO ANÁLISE FOCADA DE MÉTRICAS DE DESEMPENHO


--- [Análise de Risco: % PDD] ---

PDD de Referência da Carteira Total: 19.97%

--> Análise de PDD por 'Cedentes' (Coluna: 'CedenteNome')


,ValorLiquido (M),ValorPresente (M),%PDD
CedenteNome,,,
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,159.205096,198.934885,19.971253
BANCO DIGIMAIS S.A.,0.246985,0.297263,16.913659




--> Análise de PDD por 'Tipo de Contrato' (Coluna: 'TipoAtivo')


,ValorLiquido (M),ValorPresente (M),%PDD
TipoAtivo,,,
CCB,150.961799,190.277251,20.662192
CT - Contrato,8.490281,8.954897,5.188396




--> Análise de PDD por 'Ente Consignado' (Coluna: 'Convênio')


...



--> Análise de Vencidos por 'Tem Muitos Contratos' (Coluna: '_MuitosContratos')


,ValorVencido (M),ValorPresente (M),%Vencido
_MuitosContratos,,,
False,8.923531,170.629483,5.229771
True,1.281240,28.602664,4.479443




--> Análise de Vencidos por 'Tem Muitos Entes' (Coluna: '_MuitosEntes')


,ValorVencido (M),ValorPresente (M),%Vencido
_MuitosEntes,,,
True,0.001815,0.023654,7.672974
False,10.202956,199.208493,5.121748




FIM DA ANÁLISE


# Bloco 16: Cálculo da TIR (Taxa Interna de Retorno) da Carteira a Vencer
**Descrição:** Este bloco calcula a TIR mensal da carteira a vencer.
**v8:** Adiciona um tratamento para os casos em que a TIR resulta em NaN. Especificamente, se a PDD de um segmento for >= 100% do seu Valor Presente, a TIR Líquida (PDD) e a TIR completa são definidas como -100%, representando a perda total do principal.

<span style="color:#89CFF0;">**Observação:**  estamos usando valores *"Toy"*, para ilustração - isso exige valores reais</span>

In [19]:
#| classes: [fold-output] 
# ( saída recolhível)

from scipy.optimize import newton
import numpy as np
import pandas as pd

#

COST_DICT = {
    'ASSEMBLEIA. MATO GROSSO': [0.03, 2.14],
    'GOV. ALAGOAS': [0.035, 5.92],
}
DEFAULT_COST = COST_DICT.get('GOV. ALAGOAS', [0.035, 5.92])

def calculate_xirr(cash_flows, days):
    cash_flows, days = np.array(cash_flows), np.array(days)
    def npv(rate):
        if rate <= -1: return float('inf')
        return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
    try:
        return newton(npv, 0.015)
    except (RuntimeError, ValueError):
        return np.nan

#  PREP DOS DADOS
print("="*80)
print("INICIANDO CÁLCULO DA TIR ...")
print("="*80)

ref_date = df_final2['DataGeracao'].max()
print(f"Data de Referência para o cálculo: {ref_date.strftime('%d/%m/%Y')}")

try:
    df_feriados = pd.read_excel(caminho_feriados)
    holidays = pd.to_datetime(df_feriados['Data']).values.astype('datetime64[D]')
    print(f"Sucesso: {len(holidays)} feriados carregados de '{caminho_feriados}'.")
except Exception as e:
    print(f"[AVISO] Não foi possível carregar feriados. Erro: {e}")
    holidays = []

df_avencer = df_final2[df_final2['DataVencimento'] > ref_date].copy()
print(f"Total de {len(df_avencer)} parcelas a vencer consideradas na análise.")

try:
    start_dates = np.datetime64(ref_date.date())
    end_dates = df_avencer['DataVencimento'].values.astype('datetime64[D]')
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.busday_count(start_dates, end_dates, holidays=holidays)
    df_avencer = df_avencer[df_avencer['_DIAS_UTEIS_'] > 0]
except Exception as e:
    print(f"[ERRO] Não foi possível calcular os dias úteis: {e}")
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.nan

df_avencer['CustoVariavel'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[0])
df_avencer['CustoFixo'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[1])
df_avencer['CustoTotal'] = df_avencer['CustoFixo'] + (df_avencer['CustoVariavel'] * df_avencer['ValorNominal'])
df_avencer['ReceitaLiquida'] = df_avencer['ValorNominal'] - df_avencer['CustoTotal']


# CÁLCULO DA TIR #* por Variável Categórica (DIMENSAO)

dimensoes = cat_cols
all_tirs = []

segmentos_para_analise = [('Carteira Total', 'Todos')] + \
                         [(dim, seg) for dim in dimensoes for seg in df_avencer[dim].dropna().unique()]

for tipo_dimensao, segmento in segmentos_para_analise:
    if tipo_dimensao == 'Carteira Total':
        df_segmento = df_avencer
    else:
        df_segmento = df_avencer[df_avencer[tipo_dimensao] == segmento]

    if df_segmento.empty or df_segmento['_DIAS_UTEIS_'].isnull().all():
        continue

    vp_bruto = df_segmento['ValorPresente'].sum()
    tir_bruta, tir_pdd, tir_custos, tir_completa = np.nan, np.nan, np.nan, np.nan
    
    if vp_bruto > 0:
        pdd_total = df_segmento['PDDTotal'].sum()
        pdd_rate = pdd_total / vp_bruto
        
        #! TIR Bruta
        fluxos_brutos = df_segmento.groupby('_DIAS_UTEIS_')['ValorNominal'].sum()
        tir_bruta = calculate_xirr([-vp_bruto] + fluxos_brutos.values.tolist(), [0] + fluxos_brutos.index.tolist())

        #!TIR Líquida (PDD)
        fluxos_pdd = (df_segmento['ValorNominal'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_pdd = calculate_xirr([-vp_bruto] + fluxos_pdd.values.tolist(), [0] + fluxos_pdd.index.tolist())
        #* o caso de PDD >= 100%
        if pd.isna(tir_pdd) and pdd_rate >= 1:
            tir_pdd = -1.0 # Representa -100% de retorno

        #! TIR Líquida (Custos)
        fluxos_custos = df_segmento.groupby('_DIAS_UTEIS_')['ReceitaLiquida'].sum()
        tir_custos = calculate_xirr([-vp_bruto] + fluxos_custos.values.tolist(), [0] + fluxos_custos.index.tolist())

        # ! TIR Líquida (PDD & Custos)
        fluxos_completos = (df_segmento['ReceitaLiquida'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_completa = calculate_xirr([-vp_bruto] + fluxos_completos.values.tolist(), [0] + fluxos_completos.index.tolist())
        #* o caso de PDD >= 100%
        if pd.isna(tir_completa) and pdd_rate >= 1:
            tir_completa = -1.0 # Representa -100% de retorno

    all_tirs.append({
        'Dimensão': tipo_dimensao,
        'Segmento': segmento,
        'Valor Presente (M)': vp_bruto / 1e6,
        'TIR Bruta a.m. (%)': tir_bruta * 100 if pd.notna(tir_bruta) else np.nan,
        'TIR Líquida (PDD) a.m. (%)': tir_pdd * 100 if pd.notna(tir_pdd) else np.nan,
        'TIR Líquida (Custos) a.m. (%)': tir_custos * 100 if pd.notna(tir_custos) else np.nan,
        'TIR Líquida (PDD & Custos) a.m. (%)': tir_completa * 100 if pd.notna(tir_completa) else np.nan,
    })

#RESULTADO ==================================================

if all_tirs:
    df_tir_summary = pd.DataFrame(all_tirs)
    print("\n\n--- Tabela de Resumo da Taxa Interna de Retorno (TIR) ---\n")
    
    for dim in ['Carteira Total'] + dimensoes:
        df_display = df_tir_summary[df_tir_summary['Dimensão'] == dim]
        if not df_display.empty:
            print(f"--> TIR por '{dim}':")
            # Arredonda os valores para melhor visualização
            display(df_display.sort_values('Valor Presente (M)', ascending=False).drop(columns=['Dimensão']).round(4))
            print("\n")
else:
    print("\nNão foi possível calcular a TIR. Verifique se existem parcelas a vencer na carteira.")

print("="*80)
print("FIM DO CÁLCULO DE TIR")
print("="*80)

INICIANDO CÁLCULO DA TIR ...
Data de Referência para o cálculo: 14/07/2025
Sucesso: 1264 feriados carregados de 'C:\Users\Leo\Desktop\Porto_Real\portoauto\feriados_nacionais.xls'.
Total de 1482025 parcelas a vencer consideradas na análise.


--- Tabela de Resumo da Taxa Interna de Retorno (TIR) ---

--> TIR por 'Carteira Total':


,Segmento,Valor Presente (M),TIR Bruta a.m. (%),TIR Líquida (PDD) a.m. (%),TIR Líquida (Custos) a.m. (%),TIR Líquida (PDD & Custos) a.m. (%)
0,Todos,189.0274,2.6433,1.8317,2.3956,1.6168




--> TIR por 'Situacao':


,Segmento,Valor Presente (M),TIR Bruta a.m. (%),TIR Líquida (PDD) a.m. (%),TIR Líquida (Custos) a.m. (%),TIR Líquida (PDD & Custos) a.m. (%)
1,Sem cobranÃ§a,180.9845,2.6638,1.8165,2.4143,1.6014
2,Aditado,8.0429,2.1959,2.0865,1.9872,1.8816




--> TIR por 'CedenteNome':


...



--> TIR por '_MuitosEntes':


,Segmento,Valor Presente (M),TIR Bruta a.m. (%),TIR Líquida (PDD) a.m. (%),TIR Líquida (Custos) a.m. (%),TIR Líquida (PDD & Custos) a.m. (%)
13,False,189.0055,2.6434,1.8318,2.3957,1.6169
14,True,0.0218,1.6916,0.5234,1.4588,0.3348




--> TIR por 'Convênio':


,Segmento,Valor Presente (M),TIR Bruta a.m. (%),TIR Líquida (PDD) a.m. (%),TIR Líquida (Custos) a.m. (%),TIR Líquida (PDD & Custos) a.m. (%)
20,GOV. MARANHAO,45.6583,2.3674,1.9863,2.1617,1.7942
16,GOV. ALAGOAS,33.4171,2.7944,2.5913,2.5846,2.3889
19,GOV. GOIAS,16.0759,2.5648,0.9897,2.0985,0.6047
36,PREF. BARREIRAS,9.3457,2.6163,2.5933,2.4046,2.3825
25,GOV. RIO GRANDE DO NORTE,6.5879,2.3426,1.9265,2.0014,1.6039
41,PREF. CABO FRIO,5.9408,2.9877,-0.1745,2.7573,-0.3078
45,PREF. CAMPOS DOS GOYTACAZES,5.8646,3.0381,1.4367,2.7092,1.1733
35,PREF. BALSAS,5.1104,3.0577,3.0356,2.7566,2.7355
28,GOV. TOCANTINS,4.6061,3.0045,1.1637,2.7381,0.9723
49,PREF. CASTANHAL,3.9727,2.8474,-1.1073,2.5546,-1.2923




FIM DO CÁLCULO DE TIR


# BLOCO <span style="color:red;"> NOVO !!</span>
 <span style="color:yellow;"> Descrição: exportando tabelas em excel integrando análises </span>


In [20]:
# intro
import pandas as pd
import numpy as np
import os
from scipy.optimize import brentq # Usaremos o solver robusto 'brentq'


cat_cols = [
    'Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial',
    'TipoAtivo', '_MuitosContratos', '_MuitosEntes', 'Convênio',
    '_SacadoBMP', '_NIVEL', '_PREV', '_GENERICO'
]

dimensoes_analise = {
    'Cedentes': 'CedenteNome',
    'Tipo de Contrato': 'TipoAtivo',
    'Ente Consignado': 'Convênio',
    'Situação': 'Situacao',
    'Tipo de Pessoa Sacado':'SAC_TIPO_PESSOA',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes',
    'Sacado é BMP': '_SacadoBMP',
    'Nível do Ente': '_NIVEL',
    'Previdência': '_PREV',
    'Ente Genérico': '_GENERICO'
}

COST_DICT = {
    'ASSEMBLEIA. MATO GROSSO': [0.03, 2.14],
    'GOV. ALAGOAS': [0.035, 5.92],
}
DEFAULT_COST = COST_DICT.get('GOV. ALAGOAS', [0.035, 5.92])

output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\metricas_tabelas'
output_filename = os.path.join(output_path, 'analise_metricas_consolidadas.xlsx')

os.makedirs(output_path, exist_ok=True)
print(f"Arquivos de saída serão salvos em: {output_filename}")


#***************************
#***********   DADOS
#****************************


print("\n" + "="*80)
print("INICIANDO PREPARAÇÃO E ENRIQUECIMENTO DOS DADOS")
print("="*80)

# Criar coluna p sacado BMP
try:
    mask_bmp = df_final2['SacadoCnpjCpf'] == '34.337.707/0001-00'
    df_final2['_SacadoBMP'] = mask_bmp
    print("Coluna '_SacadoBMP' criada com sucesso.")
except KeyError:
    print("[AVISO] Coluna 'SacadoCnpjCpf' não encontrada.")

# Mapear Entes
try:
    path_map_entes = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\MAP_ENTES.xlsx'
    df_map_entes = pd.read_excel(path_map_entes)
    map_nivel = dict(zip(df_map_entes['NOME'], df_map_entes['_NIVEL']))
    map_prev = dict(zip(df_map_entes['NOME'], df_map_entes['_PREV']))
    map_generic = dict(zip(df_map_entes['NOME'], df_map_entes['_GENERICO']))
    df_final2['_NIVEL'] = df_final2['Convênio'].map(map_nivel)
    df_final2['_PREV'] = df_final2['Convênio'].map(map_prev)
    df_final2['_GENERICO'] = df_final2['Convênio'].map(map_generic)
    print("Colunas de mapeamento de entes criadas com sucesso.")
except Exception as e:
    print(f"[AVISO] Falha ao processar o mapeamento de entes: {e}")


# ==============================================================================
# TIR com brentq
# ==============================================================================
def calculate_xirr(cash_flows, days):
    """
    divide a procura da tir em uma parte padrão, e entao em uma faixa possivelmente negativa 
    """
    # descobri aqui: https://stackoverflow.com/questions/8919718/financial-python-library-that-has-xirr-and-xnpv-function
    cash_flows = np.array(cash_flows)
    days = np.array(days)

    def npv(rate):
        if rate <= -1: return float('inf')
        # convenção de 21 dias no mes
        return np.sum(cash_flows / (1 + rate) ** (days / 21.0))

    try:
        # Procura solução positiva (0% a 100% mensal)
        return brentq(npv, 0, 1.0)
    except ValueError:
        try:
            # FALHA: procura solução negativa (-99.99% a 0%)
            return brentq(npv, -0.9999, 0)
        except (RuntimeError, ValueError):
            # Se ainda assim não encontrar, o caso é insolúvel
            return np.nan

#***********************
#* CÁLCULO DAS MÉTRICAS 
#* DE PDD E INADIMPLÊNCIA
#***********************


print("\n" + "="*80)
print("INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA")
print("="*80)

tabelas_pdd = {}
tabelas_vencido = {}

# Risco: % PDD
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_pdd = df_final2.groupby(coluna)[['_ValorLiquido', 'ValorPresente']].sum()
    aux_pdd['%PDD'] = (1 - aux_pdd['_ValorLiquido'] / aux_pdd['ValorPresente']) * 100
    aux_pdd = aux_pdd.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorLiquido': 'ValorLiquido (M)'})
    aux_pdd[['ValorPresente (M)', 'ValorLiquido (M)']] /= 1e6
    tabelas_pdd[nome_analise] = aux_pdd

# Inadimplencia --- #* % Vencido
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_venc = df_final2.groupby(coluna)[['_ValorVencido', 'ValorPresente']].sum()
    aux_venc['%Vencido'] = (aux_venc['_ValorVencido'] / aux_venc['ValorPresente']) * 100
    aux_venc = aux_venc.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorVencido': 'ValorVencido (M)'})
    aux_venc[['ValorPresente (M)', 'ValorVencido (M)']] /= 1e6
    tabelas_vencido[nome_analise] = aux_venc

print("Métricas de PDD e Inadimplência calculadas.")

#***********************
#* TICKET MÉDIO PONDERADO
#***********************

print("\n" + "="*80)
print("INICIANDO CÁLCULO DO TICKET MÉDIO PONDERADO")
print("="*80)

tabelas_ticket = {}

# Ticket Médio >>> Ponderado pelo Valor Presente
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue

 
    df_temp = df_final2.dropna(subset=[coluna, 'ValorPresente', 'ValorNominal']) # drop pra nao dar erro no calculo 
    if df_temp.empty: continue

    # ticket médio ponderado
    # A fórmula usada foi: Soma(ValorNominal * ValorPresente) / Soma(ValorPresente)
    # https://www.youtube.com/watch?v=aGubfZdIQvI
    # equivale a np.average com pesos
    
    grouped = df_temp.groupby(coluna) # agrupo na categoria pra obter pesos e valores
    
    # lambda-fç de média ponderada
    weighted_avg_func = lambda x: np.average(x, weights=df_temp.loc[x.index, 'ValorPresente'])
    
    # media ponter na col 'ValorNominal'
    numerador = grouped.apply(lambda g: (g['ValorNominal'] * g['ValorPresente']).sum())
    denominador = grouped['ValorPresente'].sum()
    
    ticket_ponderado = numerador / denominador
    ticket_ponderado.name = "Ticket Ponderado (R$)"
    
    tabelas_ticket[nome_analise] = pd.DataFrame(ticket_ponderado)

print("<DEBUG> Cálculo de Ticket Médio Ponderado já terminado.")





#***********************
#* TIR
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)")
print("="*80)

ref_date = df_final2['DataGeracao'].max()
print(f"Data de Referência para o cálculo da TIR: {ref_date.strftime('%d/%m/%Y')}")

try:
    df_feriados = pd.read_excel(caminho_feriados)
    holidays = pd.to_datetime(df_feriados['Data']).values.astype('datetime64[D]')
    print(f"Sucesso: {len(holidays)} feriados carregados.")
except Exception as e:
    print(f"[AVISO] Não foi possível carregar feriados: {e}")
    holidays = []

df_avencer = df_final2[df_final2['DataVencimento'] > ref_date].copy()
try:
    start_dates = np.datetime64(ref_date.date())
    end_dates = df_avencer['DataVencimento'].values.astype('datetime64[D]')
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.busday_count(start_dates, end_dates, holidays=holidays)
    df_avencer = df_avencer[df_avencer['_DIAS_UTEIS_'] > 0]
except Exception as e:
    print(f"[ERRO] Falha ao calcular dias úteis: {e}")
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.nan

df_avencer['CustoVariavel'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[0])
df_avencer['CustoFixo'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[1])
df_avencer['CustoTotal'] = df_avencer['CustoFixo'] + (df_avencer['CustoVariavel'] * df_avencer['ValorNominal'])
df_avencer['ReceitaLiquida'] = df_avencer['ValorNominal'] - df_avencer['CustoTotal']

all_tirs = []
segmentos_para_analise = [('Carteira Total', 'Todos')] + \
                         [(col, seg) for col in cat_cols if col in df_avencer.columns for seg in df_avencer[col].dropna().unique()]

for tipo_dimensao, segmento in segmentos_para_analise:
    df_segmento = df_avencer if tipo_dimensao == 'Carteira Total' else df_avencer[df_avencer[tipo_dimensao] == segmento]
    if df_segmento.empty or df_segmento['_DIAS_UTEIS_'].isnull().all(): continue

    vp_bruto = df_segmento['ValorPresente'].sum()
    tir_bruta, tir_pdd, tir_custos, tir_completa = np.nan, np.nan, np.nan, np.nan
    
    if vp_bruto > 0:
        pdd_rate = df_segmento['PDDTotal'].sum() / vp_bruto
        
        fluxos_brutos = df_segmento.groupby('_DIAS_UTEIS_')['ValorNominal'].sum()
        tir_bruta = calculate_xirr([-vp_bruto] + fluxos_brutos.values.tolist(), [0] + fluxos_brutos.index.tolist())

        fluxos_pdd = (df_segmento['ValorNominal'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_pdd = calculate_xirr([-vp_bruto] + fluxos_pdd.values.tolist(), [0] + fluxos_pdd.index.tolist())

        fluxos_custos = df_segmento.groupby('_DIAS_UTEIS_')['ReceitaLiquida'].sum()
        tir_custos = calculate_xirr([-vp_bruto] + fluxos_custos.values.tolist(), [0] + fluxos_custos.index.tolist())

        fluxos_completos = (df_segmento['ReceitaLiquida'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_completa = calculate_xirr([-vp_bruto] + fluxos_completos.values.tolist(), [0] + fluxos_completos.index.tolist())

    all_tirs.append({
        'DimensaoColuna': tipo_dimensao,
        'Segmento': segmento,
        'Valor Presente TIR (M)': vp_bruto / 1e6,
        'TIR Bruta a.m. (%)': tir_bruta * 100 if pd.notna(tir_bruta) else np.nan,
        'TIR Líquida (PDD) a.m. (%)': tir_pdd * 100 if pd.notna(tir_pdd) else np.nan,
        'TIR Líquida (Custos) a.m. (%)': tir_custos * 100 if pd.notna(tir_custos) else np.nan,
        'TIR Líquida (PDD & Custos) a.m. (%)': tir_completa * 100 if pd.notna(tir_completa) else np.nan,
    })

df_tir_summary = pd.DataFrame(all_tirs)


#! ATENÇÃO: essa parte remove qualquer NaN da tabela. Os erros atuais são: 
#fixme      >> Diagnóstico:  - Fluxo de Caixa Inválido. 
#fixme     A soma das receitas líquidas futuras (R$ 0.00) não supera
#fixme      o investimento inicial (R$ 2,906,474.97).


#  segurança final para garantir que nenhuma célula fique vazia
tir_cols_to_fill = [col for col in df_tir_summary.columns if 'TIR' in col]
df_tir_summary[tir_cols_to_fill] = df_tir_summary[tir_cols_to_fill].fillna(-100.0)
print("Cálculo de TIR concluído.")



#***********************
#* EXPORTAÇÃO PARA EXCEL
#***********************
print("\n" + "="*80)
print("UNIFICANDO MÉTRICAS E GERANDO ARQUIVO EXCEL")
print("="*80)

with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    for nome_analise, coluna in dimensoes_analise.items():
        if coluna not in df_final2.columns: continue
        
        print(f"--> Proc. e unificando dados para a categoria: '{nome_analise}'")
        
        df_pdd = tabelas_pdd.get(nome_analise)
        df_venc = tabelas_vencido.get(nome_analise)
        df_ticket = tabelas_ticket.get(nome_analise) 
        df_tir = df_tir_summary[df_tir_summary['DimensaoColuna'] == coluna].set_index('Segmento')

        # NOTE 'how=outer' para que as categorias nao sejam perdiadas
        df_final = df_pdd.join(df_venc.drop(columns=['ValorPresente (M)']), how='outer')
        
        if df_ticket is not None:
             df_final = df_final.join(df_ticket, how='outer')

        df_final = df_final.join(df_tir.drop(columns=['DimensaoColuna']), how='outer')
        df_final.index.name = nome_analise
        df_final.reset_index(inplace=True)
        
        df_final = df_final.drop(columns=['ValorVencido (M)', 'Valor Presente TIR (M)'], errors='raise')

        #? ORDEM DAS COLUNAS
        colunas_ordem = [nome_analise, 'ValorPresente (M)', 'ValorLiquido (M)', 'Ticket Ponderado (R$)', '%PDD', '%Vencido']
        # Adic as cols de TIR
        colunas_tir_existentes = [col for col in df_tir.columns if col in df_final.columns and 'TIR' in col]
        colunas_finais = colunas_ordem + colunas_tir_existentes
        outras_colunas = [col for col in df_final.columns if col not in colunas_finais]
        """Pega quaisquer outras colunas que possam ter sobrado para não perdê-las"""
        
        df_final = df_final[colunas_finais + outras_colunas]        
        df_final = df_final.sort_values('ValorPresente (M)', ascending=False).reset_index(drop=True)
        df_final.to_excel(writer, sheet_name=nome_analise, index=False)
        
print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA COM SUCESSO!")
print(f"O arquivo consolidado foi salvo em: {output_filename}")
print("="*80)

Arquivos de saída serão salvos em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\metricas_tabelas\analise_metricas_consolidadas.xlsx

INICIANDO PREPARAÇÃO E ENRIQUECIMENTO DOS DADOS
Coluna '_SacadoBMP' criada com sucesso.
Colunas de mapeamento de entes criadas com sucesso.

INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA
Métricas de PDD e Inadimplência calculadas.

INICIANDO CÁLCULO DO TICKET MÉDIO PONDERADO


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\2347430592.py:163: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  numerador = grouped.apply(lambda g: (g['ValorNominal'] * g['ValorPresente']).sum())
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\2347430592.py:163: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  numerador = grouped.apply(lambda g: (g['ValorNominal'] * g['ValorPresente']).sum())
C:\Users\Leo\App

<DEBUG> Cálculo de Ticket Médio Ponderado já terminado.

INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)
Data de Referência para o cálculo da TIR: 14/07/2025
Sucesso: 1264 feriados carregados.


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\2347430592.py:88: RuntimeWarning: divide by zero encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\2347430592.py:88: RuntimeWarning: overflow encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\2347430592.py:88: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))


Cálculo de TIR concluído.

UNIFICANDO MÉTRICAS E GERANDO ARQUIVO EXCEL
--> Proc. e unificando dados para a categoria: 'Cedentes'
--> Proc. e unificando dados para a categoria: 'Tipo de Contrato'
--> Proc. e unificando dados para a categoria: 'Ente Consignado'
--> Proc. e unificando dados para a categoria: 'Situação'
--> Proc. e unificando dados para a categoria: 'Tipo de Pessoa Sacado'
--> Proc. e unificando dados para a categoria: 'Pagamento Parcial'
--> Proc. e unificando dados para a categoria: 'Tem Muitos Contratos'
--> Proc. e unificando dados para a categoria: 'Tem Muitos Entes'
--> Proc. e unificando dados para a categoria: 'Sacado é BMP'
--> Proc. e unificando dados para a categoria: 'Nível do Ente'
--> Proc. e unificando dados para a categoria: 'Previdência'
--> Proc. e unificando dados para a categoria: 'Ente Genérico'

ANÁLISE CONCLUÍDA COM SUCESSO!
O arquivo consolidado foi salvo em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\metricas_tabelas\analise_metricas_

In [21]:
# ==============================================================================
# =====================  CÉLULA DE DEPURAÇÃO DO TICKET =========================
# ==============================================================================

print("--- Depuração: Verificando o Cálculo do Ticket Ponderado ---")
print("O objetivo é recalcular o ticket para UM segmento e comparar com o relatório.\n")


# --- PASSO 1: Escolha um segmento para verificar ---
# Altere as duas variáveis abaixo para escolher o que quer depurar.
#
# Dica: Para verificar o segundo caso que você me passou, escolha a análise
# 'Ente Consignado' e o nome de um dos entes que compõe aquele grupo.
# Você pode pegar um nome exato da coluna 'Convênio' do seu dataframe.

nome_da_analise_a_verificar = 'Ente Consignado' # Ex: 'Ente Consignado', 'Tipo de Contrato'
coluna_correspondente = dimensoes_analise[nome_da_analise_a_verificar] # Pega o nome real da coluna, ex: 'Convênio'

# Escolha um valor específico dentro dessa coluna para analisar
# !! IMPORTANTE !!: Substitua 'NOME_DO_ENTE_EXEMPLO' por um valor real da sua coluna.
# Por exemplo: 'GOVERNO DO ESTADO DE MINAS GERAIS' ou 'SIAPE'
valor_do_segmento_a_verificar = 'GOV. MARANHAO' 


# --- PASSO 2: Isolar os dados apenas para o segmento escolhido ---
print(f"Analisando o segmento: '{valor_do_segmento_a_verificar}' da dimensão '{nome_da_analise_a_verificar}'\n")

# Filtra o DataFrame principal para conter apenas os dados do nosso alvo
df_segmento = df_final2[df_final2[coluna_correspondente] == valor_do_segmento_a_verificar].copy()

# Verificação básica
if df_segmento.empty:
    print(f"AVISO: Nenhum dado encontrado para o segmento '{valor_do_segmento_a_verificar}'.")
    print("Verifique se o nome foi digitado corretamente.")
else:
    # --- PASSO 3: Recalcular o Ticket Ponderado manualmente ---
    print("--- Cálculo Manual Passo a Passo ---")

    # A fórmula é: Soma(ValorNominal * ValorPresente) / Soma(ValorPresente)
    numerador = (df_segmento['ValorNominal'] * df_segmento['ValorPresente']).sum()
    denominador = df_segmento['ValorPresente'].sum()
    
    # Prevenção de divisão por zero
    if denominador > 0:
        ticket_manual = numerador / denominador
    else:
        ticket_manual = 0
        
    print(f"Total de Contratos no Segmento..: {len(df_segmento):,}")
    print(f"Numerador (Soma de VN * VP)......: R$ {numerador:,.2f}")
    print(f"Denominador (Soma de VP).........: R$ {denominador:,.2f}")
    print("--------------------------------------------------")
    print(f"TICKET CALCULADO MANUALMENTE.....: R$ {ticket_manual:,.2f}")
    print("--------------------------------------------------\n")


    # --- PASSO 4: Buscar o valor original que foi para o relatório ---
    print("--- Comparando com o Valor do Relatório ---")
    
    # Pega a tabela de tickets da análise correspondente
    df_ticket_relatorio = tabelas_ticket.get(nome_da_analise_a_verificar)
    
    if df_ticket_relatorio is not None and valor_do_segmento_a_verificar in df_ticket_relatorio.index:
        # Pega o valor específico do nosso segmento
        ticket_relatorio = df_ticket_relatorio.loc[valor_do_segmento_a_verificar, 'Ticket Ponderado (R$)']
        print(f"TICKET REGISTRADO NO RELATÓRIO...: R$ {ticket_relatorio:,.2f}\n")

        # --- PASSO 5: Conclusão da Verificação ---
        print("--- Conclusão ---")
        # np.isclose é a forma correta de comparar números de ponto flutuante
        if np.isclose(ticket_manual, ticket_relatorio):
            print("✅ CONSISTENTE. O cálculo manual é igual ao valor que foi para o relatório.")
            print("A variação de ticket entre abas diferentes é esperada, pois reflete a realidade de cada segmento.")
        else:
            print(f"❌ ALERTA DE INCONSISTÊNCIA. O valor manual (R$ {ticket_manual:,.2f}) é diferente do relatório (R$ {ticket_relatorio:,.2f}).")
            
    else:
        print(f"AVISO: Não foi possível encontrar o valor do ticket para '{valor_do_segmento_a_verificar}' no relatório original.")

--- Depuração: Verificando o Cálculo do Ticket Ponderado ---
O objetivo é recalcular o ticket para UM segmento e comparar com o relatório.

Analisando o segmento: 'GOV. MARANHAO' da dimensão 'Ente Consignado'

--- Cálculo Manual Passo a Passo ---
Total de Contratos no Segmento..: 314,033
Numerador (Soma de VN * VP)......: R$ 26,206,291,395.79
Denominador (Soma de VP).........: R$ 46,568,144.99
--------------------------------------------------
TICKET CALCULADO MANUALMENTE.....: R$ 562.75
--------------------------------------------------

--- Comparando com o Valor do Relatório ---
TICKET REGISTRADO NO RELATÓRIO...: R$ 562.75

--- Conclusão ---
✅ CONSISTENTE. O cálculo manual é igual ao valor que foi para o relatório.
A variação de ticket entre abas diferentes é esperada, pois reflete a realidade de cada segmento.


In [22]:
print("--- Verificando cálculo ---\n")

dim = 'Ente Consignado'
col = dimensoes_analise[dim]
seg = 'GOV. MARANHAO'

print(f"Segmento: '{seg}' | Dimensão: '{dim}'\n")

df_seg = df_final2[df_final2[col] == seg].copy()

if df_seg.empty:
    print(f"Sem dados para '{seg}'.")
else:
    print("--- Cálculo ---")

    num = (df_seg['ValorNominal'] * df_seg['ValorPresente']).sum()
    den = df_seg['ValorPresente'].sum()
    
    ticket_calc = num / den if den > 0 else 0

    print(f"Qtd: {len(df_seg):,}")
    print(f"Num: R$ {num:,.2f}")
    print(f"Den: R$ {den:,.2f}")
    print(f"Ticket: R$ {ticket_calc:,.2f}\n")

    print("--- Comparando ---")

    df_rel = tabelas_ticket.get(dim)

    if df_rel is not None and seg in df_rel.index:
        ticket_rel = df_rel.loc[seg, 'Ticket Ponderado (R$)']
        print(f"Relatório: R$ {ticket_rel:,.2f}\n")

        print("--- Final ---")
        if np.isclose(ticket_calc, ticket_rel):
            print("✓ OK. Mesmos valores.")
        else:
            print(f"✗ Diferença. Manual: R$ {ticket_calc:,.2f} | Relatório: R$ {ticket_rel:,.2f}")
    else:
        print(f"Ticket de '{seg}' não encontrado.")


--- Verificando cálculo ---

Segmento: 'GOV. MARANHAO' | Dimensão: 'Ente Consignado'

--- Cálculo ---
Qtd: 314,033
Num: R$ 26,206,291,395.79
Den: R$ 46,568,144.99
Ticket: R$ 562.75

--- Comparando ---
Relatório: R$ 562.75

--- Final ---
✓ OK. Mesmos valores.


# BLOCO NOVO: Nomeando Entes

In [23]:
# intro
import pandas as pd
import numpy as np
import os
from scipy.optimize import brentq # Usaremos o solver robusto 'brentq'

# ATUALIZADO: Substituído '_PopulacaoFaixa' pelas duas novas colunas
cat_cols = [
    'Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial',
    'TipoAtivo', '_MuitosContratos', '_MuitosEntes', 'Convênio',
    '_SacadoBMP', '_NIVEL', '_PREV', '_GENERICO',
    '_CAPAG', '_FaixaPop_Mun', '_FaixaPop_Est', '_UF' # Novas dimensões separadas
]

dimensoes_analise = {
    'Cedentes': 'CedenteNome',
    'Tipo de Contrato': 'TipoAtivo',
    'Ente Consignado': 'Convênio',
    'Situação': 'Situacao',
    'Tipo de Pessoa Sacado':'SAC_TIPO_PESSOA',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes',
    'Sacado é BMP': '_SacadoBMP',
    'Nível do Ente': '_NIVEL',
    'Previdência': '_PREV',
    'Ente Genérico': '_GENERICO'
}

COST_DICT = {
    'ASSEMBLEIA. MATO GROSSO': [0.03, 2.14],
    'GOV. ALAGOAS': [0.035, 5.92],
}
DEFAULT_COST = COST_DICT.get('GOV. ALAGOAS', [0.035, 5.92])

output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\metricas_tabelas'
output_filename = os.path.join(output_path, 'analise_metricas_consolidadas.xlsx')

os.makedirs(output_path, exist_ok=True)
print(f"Arquivos de saída serão salvos em: {output_filename}")


#***************************
#*********** DADOS
#****************************


print("\n" + "="*80)
print("INICIANDO PREPARAÇÃO E ENRIQUECIMENTO DOS DADOS")
print("="*80)

# Criar coluna p sacado BMP
try:
    mask_bmp = df_final2['SacadoCnpjCpf'] == '34.337.707/0001-00'
    df_final2['_SacadoBMP'] = mask_bmp
    print("Coluna '_SacadoBMP' criada com sucesso.")
except KeyError:
    print("[AVISO] Coluna 'SacadoCnpjCpf' não encontrada.")

# Mapear Entes
try:
    path_map_entes = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\MAP_ENTES.xlsx'
    df_map_entes = pd.read_excel(path_map_entes)
    map_nivel = dict(zip(df_map_entes['NOME'], df_map_entes['_NIVEL']))
    map_prev = dict(zip(df_map_entes['NOME'], df_map_entes['_PREV']))
    map_generic = dict(zip(df_map_entes['NOME'], df_map_entes['_GENERICO']))
    df_final2['_NIVEL'] = df_final2['Convênio'].map(map_nivel)
    df_final2['_PREV'] = df_final2['Convênio'].map(map_prev)
    df_final2['_GENERICO'] = df_final2['Convênio'].map(map_generic)
    print("Colunas de mapeamento de entes criadas com sucesso.")
except Exception as e:
    print(f"[AVISO] Falha ao processar o mapeamento de entes: {e}")
# ADICIONE ESTA LINHA AQUI:
print(f"[DIAGNÓSTICO] Valores únicos encontrados na coluna _NIVEL: {df_final2['_NIVEL'].unique()}")

#****************
#* funcao usada em montar quintis
#***************
def formatar_pop(n):
    """vou formatar para notacao de engenharia"""
    
    # ===== CORREÇÃO ADICIONADA AQUI =====
    # Se o valor de entrada for nulo ou NaN, retorna 'N/D' (Não Disponível)
    if pd.isna(n):
        return "N/D"
    # ===== FIM DA CORREÇÃO =====
    
    n = float(n)
    if n >= 1_000_000:
        return f'{n / 1_000_000:.1f}M'.replace('.0M', 'M')
    if n >= 1_000:
        return f'{n / 1_000:.0f}k'
    return str(int(n))

# ==============================================================================
# BLOCO DE ENRIQUECIMENTO UNIFICADO E CORRIGIDO
# ==============================================================================
print("\n" + "="*80)
print("INICIANDO ENRIQUECimento DE DADOS (UF, CAPAG, POPULAÇÃO E QUINTIS SEPARADOS)")
print("="*80)

try:
    # 1. LIMPEZA PARA RE-EXECUÇÃO (Idempotência)
    # ATUALIZADO: Inclui as novas colunas a serem criadas
    cols_a_criar = ['_UF', '_CAPAG', '_FaixaPop_Mun', '_FaixaPop_Est', 'Convênio_normalized', 'populacao']
    cols_para_remover = [col for col in cols_a_criar if col in df_final2.columns]
    if cols_para_remover:
        print(f"[OBS] Removendo colunas de uma execução anterior: {cols_para_remover}")
        df_final2 = df_final2.drop(columns=cols_para_remover)

    # 2. CARREGAR E PREPARAR DADOS EXTERNOS
    path_relacoes = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\relacoes.csv'
    df_relacoes = pd.read_csv(path_relacoes, sep=';')
    print(f"Arquivo de relações '{path_relacoes}' carregado com sucesso.")
    df_relacoes_unico = df_relacoes.sort_values('populacao', ascending=False).drop_duplicates(subset='Convênio', keep='first').copy()


    # 3. ENRIQUECER df_final2 com UF, CAPAG e População
    df_final2 = pd.merge(
        df_final2,
        df_relacoes_unico[['Convênio', 'UF', 'CAPAG', 'populacao']],
        on='Convênio',
        how='left'
    )
    df_final2['_UF'] = df_final2.pop('UF').fillna('Não Informado')
    df_final2['_CAPAG'] = df_final2.pop('CAPAG').fillna('Não Informado')
    print("Enriquecimento com UF, CAPAG e População concluído.")

    # 4. CRIAR FAIXAS SEPARADAS PARA MUNICÍPIOS E ESTADOS
    # Substitua a função inteira por esta versão corrigida:
    def calcular_faixas_para_nivel(df_nivel, df_rel):
        """
        Calcula os quintis de VP e os labels de faixa populacional para um subset do DataFrame.
        Recebe um DataFrame filtrado (só municípios ou só estados).
        Retorna um dicionário mapeando cada convênio ao seu respectivo label de faixa.
        """
        if df_nivel.empty:
            print(f"Nenhum dado encontrado para este nível. Pulando.")
            return {}

        vp_por_convenio = df_nivel.groupby('Convênio')['ValorPresente'].sum().sort_values()
        if vp_por_convenio.sum() == 0: return {}

        vp_cumulativo = vp_por_convenio.cumsum()
        vp_total = vp_por_convenio.sum()

        limites = [0] + [vp_total * q for q in [0.2, 0.4, 0.6, 0.8]] + [vp_total + 1]
        
        # ===== CORREÇÃO APLICADA AQUI =====
        # A lógica agora é len(limites) - 1 para gerar o número correto de labels (5).
        labels_base = [f'{chr(ord("A") + i)}' for i in range(len(limites) - 1)]
        # ===== FIM DA CORREÇÃO =====

        quintil_por_convenio = pd.cut(vp_cumulativo, bins=limites, labels=labels_base, include_lowest=True)

        df_quintil_temp = quintil_por_convenio.reset_index(name='QuintilBase')
        df_pop_e_quintil = pd.merge(df_quintil_temp, df_rel, on='Convênio', how='left').dropna(subset=['QuintilBase', 'populacao'])
        
        pop_ranges = df_pop_e_quintil.groupby('QuintilBase').agg(pop_min=('populacao', 'min'), pop_max=('populacao', 'max'))

        mapa_label_final = {}
        for quintil_base, row in pop_ranges.iterrows():
            min_fmt = formatar_pop(row['pop_min'])
            max_fmt = formatar_pop(row['pop_max'])
            label_final = f"{quintil_base}. Pop: {min_fmt}" if min_fmt == max_fmt else f"{quintil_base}. Pop: {min_fmt} a {max_fmt}"
            mapa_label_final[quintil_base] = label_final

        return quintil_por_convenio.map(mapa_label_final).to_dict()

    # Para estas, usando os nomes corretos que descobrimos:
    df_municipais = df_final2[df_final2['_NIVEL'] == 'MUNICIPIO']
    df_estaduais = df_final2[df_final2['_NIVEL'] == 'ESTADO']
    
    print("\n--- Processando Convênios Municipais ---")
    mapa_municipais = calcular_faixas_para_nivel(df_municipais, df_relacoes_unico)
    print("\n--- Processando Convênios Estaduais ---")
    mapa_estaduais = calcular_faixas_para_nivel(df_estaduais, df_relacoes_unico)

    df_final2['_FaixaPop_Mun'] = df_final2['Convênio'].map(mapa_municipais)
    df_final2['_FaixaPop_Est'] = df_final2['Convênio'].map(mapa_estaduais)

    print("\nCriação de faixas populacionais separadas concluída.")

except FileNotFoundError:
    print(f"[ERRO GRAVE] O arquivo de relações não foi encontrado em '{path_relacoes}'.")
except Exception as e:
    print(f"[ERRO] Falha inesperada durante o enriquecimento: {e}")

# 5. ATUALIZAR O DICIONÁRIO DE ANÁLISE
dimensoes_analise.update({
    'CAPAG': '_CAPAG',
    'Faixa Pop. Municipal': '_FaixaPop_Mun',
    'Faixa Pop. Estadual': '_FaixaPop_Est',
    'UF': '_UF'
})
print("\nDicionário 'dimensoes_analise' atualizado com as novas chaves separadas.")
print("="*80)


#******************
#* TIR com brentq
#*******************
def calculate_xirr(cash_flows, days):
    cash_flows = np.array(cash_flows)
    days = np.array(days)
    def npv(rate):
        if rate <= -1: return float('inf')
        with np.errstate(divide='ignore', over='ignore'):
            return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
    try:
        return brentq(npv, 0, 1.0)
    except ValueError:
        try:
            return brentq(npv, -0.9999, 0)
        except (RuntimeError, ValueError):
            return np.nan

#***********************
#* CÁLCULO DAS MÉTRICAS
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA")
print("="*80)

tabelas_pdd = {}
tabelas_vencido = {}

# Risco: % PDD
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_pdd = df_final2.groupby(coluna, observed=False)[['_ValorLiquido', 'ValorPresente']].sum()
    aux_pdd['%PDD'] = (1 - aux_pdd['_ValorLiquido'] / aux_pdd['ValorPresente']) * 100
    aux_pdd = aux_pdd.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorLiquido': 'ValorLiquido (M)'})
    aux_pdd[['ValorPresente (M)', 'ValorLiquido (M)']] /= 1e6
    tabelas_pdd[nome_analise] = aux_pdd

# Inadimplencia --- #* % Vencido
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_venc = df_final2.groupby(coluna, observed=False)[['_ValorVencido', 'ValorPresente']].sum()
    aux_venc['%Vencido'] = (aux_venc['_ValorVencido'] / aux_venc['ValorPresente']) * 100
    aux_venc = aux_venc.rename(columns={'ValorPresente': 'ValorPresente (M)', '_ValorVencido': 'ValorVencido (M)'})
    aux_venc[['ValorPresente (M)', 'ValorVencido (M)']] /= 1e6
    tabelas_vencido[nome_analise] = aux_venc

print("Métricas de PDD e Inadimplência calculadas.")

#***********************
#* TICKET MÉDIO PONDERADO
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DO TICKET MÉDIO PONDERADO")
print("="*80)

tabelas_ticket = {}

for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    df_temp = df_final2.dropna(subset=[coluna, 'ValorPresente', 'ValorNominal'])
    if df_temp.empty: continue
    grouped = df_temp.groupby(coluna, observed=False)
    numerador = grouped.apply(lambda g: (g['ValorNominal'] * g['ValorPresente']).sum(), include_groups=False)
    denominador = grouped['ValorPresente'].sum()
    ticket_ponderado = (numerador / denominador).replace([np.inf, -np.inf], 0)
    ticket_ponderado.name = "Ticket Ponderado (R$)"
    tabelas_ticket[nome_analise] = pd.DataFrame(ticket_ponderado)

print("Cálculo de Ticket Médio Ponderado concluído.")

#***********************
#* TIR
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)")
print("="*80)

ref_date = df_final2['DataGeracao'].max()
print(f"Data de Referência para o cálculo da TIR: {ref_date.strftime('%d/%m/%Y')}")

try:
    df_feriados = pd.read_excel(caminho_feriados)
    holidays = pd.to_datetime(df_feriados['Data']).values.astype('datetime64[D]')
    print(f"Sucesso: {len(holidays)} feriados carregados.")
except Exception as e:
    print(f"[AVISO] Não foi possível carregar feriados: {e}")
    holidays = []

df_avencer = df_final2[df_final2['DataVencimento'] > ref_date].copy()
try:
    start_dates = np.datetime64(ref_date.date())
    end_dates = df_avencer['DataVencimento'].values.astype('datetime64[D]')
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.busday_count(start_dates, end_dates, holidays=holidays)
    df_avencer = df_avencer[df_avencer['_DIAS_UTEIS_'] > 0]
except Exception as e:
    print(f"[ERRO] Falha ao calcular dias úteis: {e}")
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.nan

df_avencer['CustoVariavel'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[0])
df_avencer['CustoFixo'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[1])
df_avencer['CustoTotal'] = df_avencer['CustoFixo'] + (df_avencer['CustoVariavel'] * df_avencer['ValorNominal'])
df_avencer['ReceitaLiquida'] = df_avencer['ValorNominal'] - df_avencer['CustoTotal']

all_tirs = []
segmentos_para_analise = [('Carteira Total', 'Todos')] + \
                         [(col, seg) for col in cat_cols if col in df_avencer.columns for seg in df_avencer[col].dropna().unique()]

for tipo_dimensao, segmento in segmentos_para_analise:
    df_segmento = df_avencer if tipo_dimensao == 'Carteira Total' else df_avencer[df_avencer[tipo_dimensao] == segmento]
    if df_segmento.empty or df_segmento['_DIAS_UTEIS_'].isnull().all(): continue

    vp_bruto = df_segmento['ValorPresente'].sum()
    tir_bruta, tir_pdd, tir_custos, tir_completa = np.nan, np.nan, np.nan, np.nan
    
    if vp_bruto > 0:
        pdd_rate = df_segmento['PDDTotal'].sum() / vp_bruto
        
        fluxos_brutos = df_segmento.groupby('_DIAS_UTEIS_', observed=False)['ValorNominal'].sum()
        tir_bruta = calculate_xirr([-vp_bruto] + fluxos_brutos.values.tolist(), [0] + fluxos_brutos.index.tolist())

        fluxos_pdd = (df_segmento['ValorNominal'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_pdd = calculate_xirr([-vp_bruto] + fluxos_pdd.values.tolist(), [0] + fluxos_pdd.index.tolist())

        fluxos_custos = df_segmento.groupby('_DIAS_UTEIS_', observed=False)['ReceitaLiquida'].sum()
        tir_custos = calculate_xirr([-vp_bruto] + fluxos_custos.values.tolist(), [0] + fluxos_custos.index.tolist())
        
        df_segmento_copy = df_segmento.copy()
        df_segmento_copy['FluxoCompleto'] = (df_segmento_copy['ValorNominal'] * (1 - df_segmento_copy['CustoVariavel'])) * (1 - pdd_rate) - df_segmento_copy['CustoFixo']
        fluxos_completos = df_segmento_copy.groupby('_DIAS_UTEIS_', observed=False)['FluxoCompleto'].sum()
        tir_completa = calculate_xirr([-vp_bruto] + fluxos_completos.values.tolist(), [0] + fluxos_completos.index.tolist())

    all_tirs.append({
        'DimensaoColuna': tipo_dimensao,
        'Segmento': segmento,
        'Valor Presente TIR (M)': vp_bruto / 1e6,
        'TIR Bruta a.m. (%)': tir_bruta * 100 if pd.notna(tir_bruta) else np.nan,
        'TIR Líquida (PDD) a.m. (%)': tir_pdd * 100 if pd.notna(tir_pdd) else np.nan,
        'TIR Líquida (Custos) a.m. (%)': tir_custos * 100 if pd.notna(tir_custos) else np.nan,
        'TIR Líquida (PDD & Custos) a.m. (%)': tir_completa * 100 if pd.notna(tir_completa) else np.nan,
    })

df_tir_summary = pd.DataFrame(all_tirs)
tir_cols_to_fill = [col for col in df_tir_summary.columns if 'TIR' in col]
df_tir_summary[tir_cols_to_fill] = df_tir_summary[tir_cols_to_fill].fillna(-100.0)
print("Cálculo de TIR concluído.")

#***********************
#* EXPORTAÇÃO PARA EXCEL
#***********************
print("\n" + "="*80)
print("UNIFICANDO MÉTRICAS E GERANDO ARQUIVO EXCEL")
print("="*80)

# ATUALIZADO: Incluídas as novas faixas para ordenação alfabética
dimensoes_ordem_alfabetica = ['Faixa Pop. Municipal', 'Faixa Pop. Estadual', 'CAPAG']

with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    for nome_analise, coluna in dimensoes_analise.items():
        if coluna not in df_final2.columns or df_final2[coluna].isnull().all(): continue
        
        print(f"--> Processando e unificando dados para a categoria: '{nome_analise}'")
        
        df_pdd = tabelas_pdd.get(nome_analise)
        df_venc = tabelas_vencido.get(nome_analise)
        df_ticket = tabelas_ticket.get(nome_analise)
        df_tir = df_tir_summary[df_tir_summary['DimensaoColuna'] == coluna].set_index('Segmento')

        # Se a tabela de pdd (base) não foi gerada, pula para a próxima dimensão
        if df_pdd is None:
            print(f"    [AVISO] Sem dados para a dimensão '{nome_analise}'. Pulando.")
            continue

        df_final = df_pdd.join(df_venc.drop(columns=['ValorPresente (M)']), how='outer')
        
        if df_ticket is not None:
            df_final = df_final.join(df_ticket, how='outer')

        df_final = df_final.join(df_tir.drop(columns=['DimensaoColuna']), how='outer')
        df_final.index.name = nome_analise
        df_final.reset_index(inplace=True)
        
        df_final = df_final.drop(columns=['ValorVencido (M)', 'Valor Presente TIR (M)'], errors='ignore')

        # ===== INÍCIO DA CORREÇÃO =====
        # Cria a ordem das colunas de forma dinâmica, verificando se elas existem
        
        colunas_ordem = [nome_analise, 'ValorLiquido (M)', 'ValorPresente (M)']
        
        # Adiciona o Ticket Ponderado apenas SE ele foi calculado e existe em df_final
        if 'Ticket Ponderado (R$)' in df_final.columns:
            colunas_ordem.append('Ticket Ponderado (R$)')
        
        colunas_ordem.extend(['%PDD', '%Vencido'])
        # ===== FIM DA CORREÇÃO =====

        colunas_tir_existentes = [col for col in df_tir.columns if col in df_final.columns and 'TIR' in col]
        colunas_finais = colunas_ordem + sorted(colunas_tir_existentes)
        
        outras_colunas = [col for col in df_final.columns if col not in colunas_finais]
        
        df_final = df_final[colunas_finais + outras_colunas]
        
        if nome_analise in dimensoes_ordem_alfabetica:
            df_final = df_final.sort_values(nome_analise, ascending=True).reset_index(drop=True)
        else:
            df_final = df_final.sort_values('ValorPresente (M)', ascending=False).reset_index(drop=True)
        
        df_final.to_excel(writer, sheet_name=nome_analise, index=False)
        
print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA COM SUCESSO!")
print(f"O arquivo consolidado foi salvo em: {output_filename}")
print("="*80)

Arquivos de saída serão salvos em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\metricas_tabelas\analise_metricas_consolidadas.xlsx

INICIANDO PREPARAÇÃO E ENRIQUECIMENTO DOS DADOS
Coluna '_SacadoBMP' criada com sucesso.
Colunas de mapeamento de entes criadas com sucesso.
[DIAGNÓSTICO] Valores únicos encontrados na coluna _NIVEL: ['ESTADO' 'MUNICIPIO']

INICIANDO ENRIQUECimento DE DADOS (UF, CAPAG, POPULAÇÃO E QUINTIS SEPARADOS)
Arquivo de relações 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\fct_consig\relacoes.csv' carregado com sucesso.
Enriquecimento com UF, CAPAG e População concluído.

--- Processando Convênios Municipais ---


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_ranges = df_pop_e_quintil.groupby('QuintilBase').agg(pop_min=('populacao', 'min'), pop_max=('populacao', 'max'))



--- Processando Convênios Estaduais ---


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_ranges = df_pop_e_quintil.groupby('QuintilBase').agg(pop_min=('populacao', 'min'), pop_max=('populacao', 'max'))



Criação de faixas populacionais separadas concluída.

Dicionário 'dimensoes_analise' atualizado com as novas chaves separadas.

INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA
Métricas de PDD e Inadimplência calculadas.

INICIANDO CÁLCULO DO TICKET MÉDIO PONDERADO
Cálculo de Ticket Médio Ponderado concluído.

INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)
Data de Referência para o cálculo da TIR: 14/07/2025
Sucesso: 1264 feriados carregados.


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))


Cálculo de TIR concluído.

UNIFICANDO MÉTRICAS E GERANDO ARQUIVO EXCEL
--> Processando e unificando dados para a categoria: 'Cedentes'
--> Processando e unificando dados para a categoria: 'Tipo de Contrato'
--> Processando e unificando dados para a categoria: 'Ente Consignado'
--> Processando e unificando dados para a categoria: 'Situação'
--> Processando e unificando dados para a categoria: 'Tipo de Pessoa Sacado'
--> Processando e unificando dados para a categoria: 'Pagamento Parcial'
--> Processando e unificando dados para a categoria: 'Tem Muitos Contratos'
--> Processando e unificando dados para a categoria: 'Tem Muitos Entes'
--> Processando e unificando dados para a categoria: 'Sacado é BMP'
--> Processando e unificando dados para a categoria: 'Nível do Ente'
--> Processando e unificando dados para a categoria: 'Previdência'
--> Processando e unificando dados para a categoria: 'Ente Genérico'
--> Processando e unificando dados para a categoria: 'CAPAG'
--> Processando e unifican

In [24]:


#***********************
#* CÁLCULO DAS MÉTRICAS
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA")
print("="*80)

# --- NOVOS NOMES DAS COLUNAS ---
vp_col_name = 'Valor Presente \n(R$ MM)'
vl_col_name = 'Valor Líquido \n(R$ MM)'
# -----------------------------

tabelas_pdd = {}
tabelas_vencido = {}

# Risco: % PDD
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_pdd = df_final2.groupby(coluna, observed=False)[['_ValorLiquido', 'ValorPresente']].sum()
    aux_pdd['%PDD'] = (1 - aux_pdd['_ValorLiquido'] / aux_pdd['ValorPresente']) * 100
    # MODIFICADO AQUI: Renomeando para os novos nomes com quebra de linha
    aux_pdd = aux_pdd.rename(columns={'ValorPresente': vp_col_name, '_ValorLiquido': vl_col_name})
    aux_pdd[[vp_col_name, vl_col_name]] /= 1e6
    tabelas_pdd[nome_analise] = aux_pdd

# Inadimplencia --- #* % Vencido
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    aux_venc = df_final2.groupby(coluna, observed=False)[['_ValorVencido', 'ValorPresente']].sum()
    aux_venc['%Vencido'] = (aux_venc['_ValorVencido'] / aux_venc['ValorPresente']) * 100
    # MODIFICADO AQUI: Renomeando para os novos nomes com quebra de linha
    aux_venc = aux_venc.rename(columns={'ValorPresente': vp_col_name, '_ValorVencido': 'ValorVencido (M)'})
    aux_venc[[vp_col_name, 'ValorVencido (M)']] /= 1e6
    tabelas_vencido[nome_analise] = aux_venc

print("Métricas de PDD e Inadimplência calculadas.")

#***********************
#* TIR
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)")
print("="*80)

ref_date = df_final2['DataGeracao'].max()
print(f"Data de Referência para o cálculo da TIR: {ref_date.strftime('%d/%m/%Y')}")

try:
    df_feriados = pd.read_excel(caminho_feriados)
    holidays = pd.to_datetime(df_feriados['Data']).values.astype('datetime64[D]')
    print(f"Sucesso: {len(holidays)} feriados carregados.")
except Exception as e:
    print(f"[AVISO] Não foi possível carregar feriados: {e}")
    holidays = []

df_avencer = df_final2[df_final2['DataVencimento'] > ref_date].copy()
try:
    start_dates = np.datetime64(ref_date.date())
    end_dates = df_avencer['DataVencimento'].values.astype('datetime64[D]')
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.busday_count(start_dates, end_dates, holidays=holidays)
    df_avencer = df_avencer[df_avencer['_DIAS_UTEIS_'] > 0]
except Exception as e:
    print(f"[ERRO] Falha ao calcular dias úteis: {e}")
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.nan

df_avencer['CustoVariavel'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[0])
df_avencer['CustoFixo'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[1])
df_avencer['CustoTotal'] = df_avencer['CustoFixo'] + (df_avencer['CustoVariavel'] * df_avencer['ValorNominal'])
df_avencer['ReceitaLiquida'] = df_avencer['ValorNominal'] - df_avencer['CustoTotal']

all_tirs = []
segmentos_para_analise = [('Carteira Total', 'Todos')] + \
                         [(col, seg) for col in cat_cols if col in df_avencer.columns for seg in df_avencer[col].dropna().unique()]

for tipo_dimensao, segmento in segmentos_para_analise:
    df_segmento = df_avencer if tipo_dimensao == 'Carteira Total' else df_avencer[df_avencer[tipo_dimensao] == segmento]
    if df_segmento.empty or df_segmento['_DIAS_UTEIS_'].isnull().all(): continue

    vp_bruto = df_segmento['ValorPresente'].sum()
    tir_bruta, tir_pdd, tir_custos, tir_completa = np.nan, np.nan, np.nan, np.nan

    if vp_bruto > 0:
        pdd_rate = df_segmento['PDDTotal'].sum() / vp_bruto
        fluxos_brutos = df_segmento.groupby('_DIAS_UTEIS_', observed=False)['ValorNominal'].sum()
        tir_bruta = calculate_xirr([-vp_bruto] + fluxos_brutos.values.tolist(), [0] + fluxos_brutos.index.tolist())
        fluxos_pdd = (df_segmento['ValorNominal'] * (1 - pdd_rate)).groupby(df_segmento['_DIAS_UTEIS_']).sum()
        tir_pdd = calculate_xirr([-vp_bruto] + fluxos_pdd.values.tolist(), [0] + fluxos_pdd.index.tolist())
        fluxos_custos = df_segmento.groupby('_DIAS_UTEIS_', observed=False)['ReceitaLiquida'].sum()
        tir_custos = calculate_xirr([-vp_bruto] + fluxos_custos.values.tolist(), [0] + fluxos_custos.index.tolist())
        df_segmento_copy = df_segmento.copy()
        df_segmento_copy['FluxoCompleto'] = (df_segmento_copy['ValorNominal'] * (1 - df_segmento_copy['CustoVariavel'])) * (1 - pdd_rate) - df_segmento_copy['CustoFixo']
        fluxos_completos = df_segmento_copy.groupby('_DIAS_UTEIS_', observed=False)['FluxoCompleto'].sum()
        tir_completa = calculate_xirr([-vp_bruto] + fluxos_completos.values.tolist(), [0] + fluxos_completos.index.tolist())

    # MODIFICADO AQUI: Usando os novos nomes para as colunas de TIR
    all_tirs.append({
        'DimensaoColuna': tipo_dimensao,
        'Segmento': segmento,
        'Valor Presente TIR (M)': vp_bruto / 1e6,
        'TIR Bruta \n(% a.m. )': tir_bruta * 100 if pd.notna(tir_bruta) else np.nan,
        'TIR Líquida de PDD \n(% a.m. )': tir_pdd * 100 if pd.notna(tir_pdd) else np.nan,
        'TIR Líquida de custos \n(% a.m. )': tir_custos * 100 if pd.notna(tir_custos) else np.nan,
        'TIR Líquida Final \n(% a.m. )': tir_completa * 100 if pd.notna(tir_completa) else np.nan,
    })

df_tir_summary = pd.DataFrame(all_tirs)
tir_cols_to_fill = [col for col in df_tir_summary.columns if 'TIR' in col]
df_tir_summary[tir_cols_to_fill] = df_tir_summary[tir_cols_to_fill].fillna(-100.0)
print("Cálculo de TIR concluído.")




INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA
Métricas de PDD e Inadimplência calculadas.

INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)
Data de Referência para o cálculo da TIR: 14/07/2025
Sucesso: 1264 feriados carregados.


C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
C:\Users\Leo\AppData\Local\Temp\ipykernel_17404\41802809.py:209: RuntimeWarning: invalid value encountered in divide
  return np.sum(cash_flows / (1 + rate) ** (days / 21.0))


Cálculo de TIR concluído.


In [25]:
# *****************************************************************************
# * BLOCO DE EXPORTAÇÃO (FINAL, COM AJUSTES FINAIS DE ESTILO)
# *****************************************************************************
import base64
import os

print("\n" + "="*80)
print("GERANDO RELATÓRIO HTML FINAL COM AJUSTES DE ESTILO")
print("="*80)

# --- 1. PREPARAÇÃO DOS ATIVOS (LOGO E DATA) ---
def encode_image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"[AVISO] Arquivo de imagem não encontrado em: {image_path}. A logo não será exibida.")
        return None

logo_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png'
logo_base64 = encode_image_to_base64(logo_path)
report_date = ref_date.strftime('%d/%m/%Y')

# --- 2. DEFINIÇÃO DO CSS COMPLETO ---

# MODIFICADO AQUI: Ajustes finais no CSS do cabeçalho
html_css = """
<style>
    /* Configurações Gerais */
    body {
        font-family: "Gill Sans MT", Arial, sans-serif;
        background-color: #f9f9f9;
        color: #313131;
        margin: 0;
        padding: 0;
    }
    .main-content {
        padding: 25px;
    }

    /* --- CABEÇALHO --- */
    header {
        background-color: #163f3f;
        color: #FFFFFF;
        padding: 20px 40px;
        display: flex;
        justify-content: space-between;
        align-items: center;
        border-bottom: 5px solid #76c6c5;
    }
    /* 3. LOGO MAIOR: Altura da logo aumentada novamente */
    header .logo img {
        height: 75px; /* Aumentado de 65px para 75px */
    }
    header .report-title {
        text-align: left;
        font-family: "Gill Sans MT", Arial, sans-serif;
    }
    header .report-title h1, header .report-title h2, header .report-title h3 {
        margin: 0;
        padding: 0;
        font-weight: normal;
    }
    /* 2. FONTE MENOR: Tamanho do título principal reduzido */
    header .report-title h1 { font-size: 1.6em; /* Reduzido de 1.8em para 1.6em */ }
    header .report-title h2 { font-size: 1.4em; color: #d0d0d0; }
    header .report-title h3 { font-size: 1.1em; color: #a0a0a0; }

    /* Estilos dos Botões e Tabelas (sem alterações) */
    .container-botoes { display: flex; flex-wrap: wrap; gap: 15px; margin-bottom: 25px; }
    .container-botoes > details { flex: 1 1 280px; border: 1px solid #76c6c5; border-radius: 8px; overflow: hidden; }
    .container-botoes > details[open] { flex-basis: 100%; }
    details summary { font-size: 1.1em; font-weight: bold; color: #FFFFFF; background-color: #163f3f; padding: 15px 20px; cursor: pointer; outline: none; list-style-type: none; }
    details summary:hover { background-color: #0e5d5f; }
    details[open] summary { background-color: #76c6c5; color: #313131; }
    details[open] summary:hover { filter: brightness(95%); }
    summary::-webkit-details-marker { display: none; }
    summary::before { content: '► '; margin-right: 8px; font-size: 0.8em;}
    details[open] summary::before { content: '▼ '; }
    details .content-wrapper { padding: 20px; background-color: #FFFFFF; }
    table.dataframe, th, td { border: 1px solid #bbbbbb; }
    table.dataframe { border-collapse: collapse; width: 100%; }
    th, td { text-align: left; padding: 10px; vertical-align: middle; }
    th { background-color: #163f3f; color: #FFFFFF; }
    tr:nth-child(even) { background-color: #eeeeee; }

    /* --- RODAPÉ --- */
    footer {
        background-color: #f0f0f0;
        color: #555555;
        font-size: 0.8em;
        line-height: 1.6;
        padding: 25px 40px;
        margin-top: 40px;
        border-top: 1px solid #dddddd;
    }
    footer .disclaimer {
        margin-top: 20px;
        font-style: italic;
        border-top: 1px solid #dddddd;
        padding-top: 20px;
    }
</style>
"""

# --- 3. CONSTRUÇÃO DO HTML COMPLETO ---

html_parts = []
html_parts.append("<!DOCTYPE html><html lang='pt-BR'><head>")
html_parts.append("<meta charset='UTF-8'><title>Análise de Desempenho - FCT Consignado II</title>")
html_parts.append(html_css)
html_parts.append("</head><body>")

# --- Adiciona o Cabeçalho ---
html_parts.append("<header>")
html_parts.append(f"""
<div class="report-title">
    <h1>Análise de desempenho</h1>
    <h2>FCT CONSIGNADO II</h2>
    <h3>{report_date}</h3>
</div>
""")
if logo_base64:
    html_parts.append(f'<div class="logo"><img src="data:image/png;base64,{logo_base64}" alt="Logo"></div>')
html_parts.append("</header>")

html_parts.append("<div class='main-content'>")
mapa_descricoes = {
    'Cedentes': 'Analisa as métricas de risco e retorno agrupadas por cada Cedente (originador) dos títulos.', 'Tipo de Contrato': 'Agrupa os dados por Tipo de Ativo (CCB, Contrato) para comparar o desempenho de cada um.', 'Ente Consignado': 'Métricas detalhadas por cada Convênio (ente público ou privado) onde a consignação é feita.', 'Situação': 'Compara o desempenho dos títulos com base na sua situação atual (ex: Aditado, Liquidado).', 'Tipo de Pessoa Sacado': 'Diferencia a análise entre sacados Pessoa Física (F) e Jurídica (J).', 'Pagamento Parcial': 'Verifica se há impacto nas métricas para títulos que aceitam pagamento parcial.', 'Tem Muitos Contratos': 'Compara sacados com um número baixo vs. alto de contratos (CCBs) ativos.', 'Tem Muitos Entes': 'Compara sacados que operam em poucos vs. múltiplos convênios (entes).', 'Sacado é BMP': 'Isola e analisa especificamente as operações com o sacado BMP S.A.', 'Nível do Ente': 'Agrupa os convênios por nível governamental (Municipal, Estadual, Federal).', 'Previdência': 'Identifica e analisa separadamente os convênios que são de regimes de previdência.', 'Ente Genérico': 'Agrupa convênios que não se encaixam em uma categoria específica.', 'CAPAG': 'Métricas baseadas na Capacidade de Pagamento (CAPAG) do município ou estado, uma nota do Tesouro Nacional.', 'Faixa Pop. Municipal': 'Agrupa os convênios municipais por faixas de população.', 'Faixa Pop. Estadual': 'Agrupa os convênios estaduais por faixas de população.', 'UF': 'Agrega todas as métricas por Unidade Federativa (Estado).'
}
html_parts.append("<div class='container-botoes'>")
dimensoes_ordem_alfabetica = ['Faixa Pop. Municipal', 'Faixa Pop. Estadual', 'CAPAG']
vp_col_name = 'Valor Presente \n(R$ MM)'
vl_col_name = 'Valor Líquido \n(R$ MM)'
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns or df_final2[coluna].isnull().all(): continue
    print(f"--> Processando e gerando HTML para o botão: '{nome_analise}'")
    df_pdd = tabelas_pdd.get(nome_analise)
    df_venc = tabelas_vencido.get(nome_analise)
    df_ticket = tabelas_ticket.get(nome_analise)
    df_tir = df_tir_summary[df_tir_summary['DimensaoColuna'] == coluna].set_index('Segmento')
    if df_pdd is None: continue
    df_final = df_pdd.join(df_venc.drop(columns=[vp_col_name]), how='outer')
    if df_ticket is not None: df_final = df_final.join(df_ticket, how='outer')
    df_final = df_final.join(df_tir.drop(columns=['DimensaoColuna']), how='outer')
    df_final.index.name = nome_analise
    df_final.reset_index(inplace=True)
    df_final = df_final.drop(columns=['ValorVencido (M)', 'Valor Presente TIR (M)'], errors='ignore')
    colunas_ordem = [nome_analise, vl_col_name, vp_col_name]
    if 'Ticket Ponderado (R$)' in df_final.columns: colunas_ordem.append('Ticket Ponderado (R$)')
    colunas_ordem.extend(['%PDD', '%Vencido'])
    colunas_tir_existentes = sorted([col for col in df_tir.columns if col in df_final.columns and 'TIR' in col])
    colunas_finais = colunas_ordem + colunas_tir_existentes
    outras_colunas = [col for col in df_final.columns if col not in colunas_finais]
    df_final = df_final[colunas_finais + outras_colunas]
    if nome_analise in dimensoes_ordem_alfabetica:
        df_final = df_final.sort_values(nome_analise, ascending=True).reset_index(drop=True)
    else:
        df_final = df_final.sort_values(vp_col_name, ascending=False).reset_index(drop=True)
    formatters = { vl_col_name: lambda x: f'{x:,.2f}', vp_col_name: lambda x: f'{x:,.2f}', 'Ticket Ponderado (R$)': lambda x: f'R$ {x:,.2f}', '%PDD': lambda x: f'{x:,.2f}%', '%Vencido': lambda x: f'{x:,.2f}%', }
    for col in colunas_tir_existentes: formatters[col] = lambda x: f'{x:,.2f}%'
    df_final.columns = [col.replace('\n', '<br>') for col in df_final.columns]
    html_parts.append("<details>")
    descricao = mapa_descricoes.get(nome_analise, 'Descrição não disponível.')
    html_parts.append(f'<summary title="{descricao}">{nome_analise}</summary>')
    html_parts.append("<div class='content-wrapper'>")
    html_table = df_final.to_html(index=False, classes='dataframe', formatters=formatters, na_rep='-', escape=False)
    html_parts.append(html_table)
    html_parts.append("</div></details>")
html_parts.append("</div>")
html_parts.append("</div>")

# --- Adiciona o Rodapé ---
footer_main_text = """
Este documento tem como objetivo apresentar uma análise de desempenho do fundo FCT Consignado II (CNPJ 52.203.615/0001-19), realizada pelo Porto Real Investimentos na qualidade de cogestora. Os prestadores de serviço do fundo são: FICTOR ASSET (Gestor), Porto Real Investimentos (Cogestor), e VÓRTX DTVM (Administrador e Custodiante).
"""
footer_disclaimer = """
Disclaimer: Este relatório foi preparado pelo Porto Real Investimentos exclusivamente para fins informativos e não constitui uma oferta de venda, solicitação de compra ou recomendação para qualquer investimento. As informações aqui contidas são baseadas em fontes consideradas confiáveis na data de sua publicação, mas não há garantia de sua precisão ou completude. Rentabilidade passada não representa garantia de rentabilidade futura.
"""
html_parts.append("<footer>")
html_parts.append(f'<p>{footer_main_text.strip()}</p>')
html_parts.append(f'<div class="disclaimer">{footer_disclaimer.strip()}</div>')
html_parts.append("</footer>")

html_parts.append("</body></html>")

# --- 4. SALVA O ARQUIVO FINAL ---
final_html_content = "\n".join(html_parts)
html_output_filename = os.path.join(output_path, 'analise_metricas_consolidadas.html')
try:
    with open(html_output_filename, 'w', encoding='utf-8') as f:
        f.write(final_html_content)
    print("\n" + "="*80)
    print("ANÁLISE CONCLUÍDA COM SUCESSO!")
    print(f"O relatório HTML final foi salvo em: {html_output_filename}")
    print("="*80)
except Exception as e:
    print(f"\n[ERRO GRAVE] Não foi possível salvar o arquivo HTML: {e}")


GERANDO RELATÓRIO HTML FINAL COM AJUSTES DE ESTILO
--> Processando e gerando HTML para o botão: 'Cedentes'
--> Processando e gerando HTML para o botão: 'Tipo de Contrato'
--> Processando e gerando HTML para o botão: 'Ente Consignado'
--> Processando e gerando HTML para o botão: 'Situação'
--> Processando e gerando HTML para o botão: 'Tipo de Pessoa Sacado'
--> Processando e gerando HTML para o botão: 'Pagamento Parcial'
--> Processando e gerando HTML para o botão: 'Tem Muitos Contratos'
--> Processando e gerando HTML para o botão: 'Tem Muitos Entes'
--> Processando e gerando HTML para o botão: 'Sacado é BMP'
--> Processando e gerando HTML para o botão: 'Nível do Ente'
--> Processando e gerando HTML para o botão: 'Previdência'
--> Processando e gerando HTML para o botão: 'Ente Genérico'
--> Processando e gerando HTML para o botão: 'CAPAG'
--> Processando e gerando HTML para o botão: 'Faixa Pop. Municipal'
--> Processando e gerando HTML para o botão: 'Faixa Pop. Estadual'
--> Processand

# Bloco 17: Visualização Gráfica da Distribuição da Carteira
**Descrição:** Gera gráficos para analisar a distribuição da carteira.
**v20:** Atende à solicitação do usuário para não mostrar nos gráficos os entes cuja TIR calculada resultou em -100%.

In [26]:
#| classes: [fold-output] 
# ( saída recolhível)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("="*80)
print("INICIANDO GERAÇÃO DE GRÁFICOS DE DISTRIBUIÇÃO")
print("="*80)



df_resumo_metricas = df_final2.groupby('Convênio').agg(
    ValorPresente=('ValorPresente', 'sum'),
    ValorLiquido=('_ValorLiquido', 'sum'),
    ValorVencido=('_ValorVencido', 'sum'),
    PDDTotal=('PDDTotal', 'sum')
).reset_index()
"""# ---  preprc do datfrm de resmo para os grafcs ---"""

df_resumo_metricas['%Vencido'] = (df_resumo_metricas['ValorVencido'] / df_resumo_metricas['ValorPresente']) * 100
df_resumo_metricas['%PDD'] = (df_resumo_metricas['PDDTotal'] / df_resumo_metricas['ValorPresente']) * 100

df_resumo_tir = df_tir_summary[df_tir_summary['Dimensão'] == 'Convênio'].copy()

df_graficos = pd.merge(
    df_resumo_metricas,
    df_resumo_tir.rename(columns={'Segmento': 'Convênio'}),
    on='Convênio',
    how='left'
)



#####################################################################

def plot_ranking_com_outros(df, metric_col, title, n=15, agg_type='sum', is_percent=False):
    """#  função de plotgm de rankng aprmrd ---"""
    df_sorted = df.dropna(subset=[metric_col]).sort_values(metric_col, ascending=False)
    
    if len(df_sorted) > n:
        df_top = df_sorted.head(n)
        df_others = df_sorted.iloc[n:]
        
        if agg_type == 'sum':
            others_value = df_others[metric_col].sum()
        else: # 'mean'
            others_value = df_others[metric_col].mean()
            
        others_name = f'Outros ({len(df_others)} entes)'
        df_outros = pd.DataFrame([{'Convênio': others_name, metric_col: others_value}])
        df_to_plot = pd.concat([df_top, df_outros], ignore_index=True)
    else:
        df_to_plot = df_sorted

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 8))
    bars = plt.barh(df_to_plot['Convênio'], df_to_plot[metric_col], color=sns.color_palette("viridis", len(df_to_plot)))
    plt.gca().invert_yaxis()
    
    for bar in bars:
        width = bar.get_width()
        if pd.isna(width): continue
        label_format = '{:,.2f}%' if is_percent else 'R$ {:,.2f}M'
        label_value = width if is_percent else width / 1e6
        plt.text(width, bar.get_y() + bar.get_height()/2, f' {label_format.format(label_value)}', va='center')
        
    plt.title(title, fontsize=16, weight='bold')
    plt.xlabel('Valor' + (' (%)' if is_percent else ''), fontsize=12)
    plt.ylabel('Ente (Convênio)', fontsize=12)
    plt.tight_layout()
    plt.show()

#? Graficos >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# *&Composição da Carteira
print("\nGerando Gráfico 1: Distribuição da Carteira por Valor Líquido...")
plot_ranking_com_outros(
    df_graficos, 'ValorLiquido',
    'Gráfico 1: Distribuição da Carteira por Valor Líquido (Top 15 + Outros)',
    n=15, agg_type='sum', is_percent=False
)

# *Ranking por Risco (% Vencido)
print("\nGerando Gráfico 2: Ranking por Risco...")
plot_ranking_com_outros(
    df_graficos, '%Vencido',
    'Gráfico 2: Top 15 Entes por % de Carteira Vencida',
    n=15, agg_type='mean', is_percent=True
)

#*Ranking por Retorno (TIR)
#>>>>>>>== obs filtra os entes com TIR de -100% antes de plotar
df_tir_filtrado = df_graficos[df_graficos['TIR Líquida (PDD & Custos) a.m. (%)'] != -100.0]
plot_ranking_com_outros(
    df_tir_filtrado, 'TIR Líquida (PDD & Custos) a.m. (%)',
    'Gráfico 3: Top 15 Entes por Retorno (TIR Líquida Completa)',
    n=15, agg_type='mean', is_percent=True
)


#* ANALISE DE RISCO vs. RETORNO (adap dashboard)
print("\nGerando Gráfico 4: Risco vs. Retorno Simplificado...")
# Preparo os dados
df_scatter = df_graficos.dropna(subset=['%Vencido', 'TIR Líquida (PDD & Custos) a.m. (%)']).copy()
df_scatter = df_scatter[
    (df_scatter['TIR Líquida (PDD & Custos) a.m. (%)'] != -100.0) &
    (np.isfinite(df_scatter['%Vencido'])) &
    (np.isfinite(df_scatter['TIR Líquida (PDD & Custos) a.m. (%)']))
]

if not df_scatter.empty:
    plt.figure(figsize=(14, 9))
    
    sns.scatterplot(
        data=df_scatter,
        x='%Vencido',
        y='TIR Líquida (PDD & Custos) a.m. (%)',
        color='darkcyan',
        alpha=0.7
    )

    mean_risk = df_scatter['%Vencido'].mean()
    mean_tir = df_scatter['TIR Líquida (PDD & Custos) a.m. (%)'].mean()
    plt.axvline(mean_risk, color='red', linestyle='--', lw=1)
    plt.axhline(mean_tir, color='red', linestyle='--', lw=1)

    plt.text(plt.xlim()[1], mean_tir, ' Média TIR', va='center', ha='right', backgroundcolor='white', color='red')
    plt.text(mean_risk, plt.ylim()[1], ' Média Risco', va='top', ha='left', backgroundcolor='white', color='red')
    
    # anotcs nos ponts para os mars ents- util para identf os ponts chve
    for i, row in df_scatter.nlargest(5, 'ValorLiquido').iterrows():
        plt.text(row['%Vencido'] + 0.1, row['TIR Líquida (PDD & Custos) a.m. (%)'], row['Convênio'], fontsize=9, ha='left')

    plt.title('Gráfico 4: Análise de Risco (% Vencido) vs. Retorno (TIR)', fontsize=16, weight='bold')
    plt.xlabel('Risco (% da Carteira Vencida)', fontsize=12)
    plt.ylabel('Retorno (TIR Líquida Completa % a.m.)', fontsize=12)
    plt.grid(True)
    plt.show()
else:
    print("Não foi possível gerar o gráfico de Risco vs. Retorno pois não há dados suficientes (ou todos foram filtrados).")

print("="*80)
print("FIM DA GERAÇÃO DE GRÁFICOS")
print("="*80)

INICIANDO GERAÇÃO DE GRÁFICOS DE DISTRIBUIÇÃO


KeyError: 'Dimensão'

# Bloco 18: Explicação dos Métodos
 <span style="color:#98FB98;"> - Como a TIR foi calculada</span>

Explicação
## Explicação Geral Sobre o Cálculo da TIR

## Metodologia de Cálculo da Taxa Interna de Retorno (TIR)

### Geral
A fórmula geral que o script resolve para encontrar a TIR é:

$$VPL = \sum_{i=0}^{n} \frac{FC_i}{(1 + \text{TIR})^{d_i/P}} = 0$$

com:
- $FC_i$: É o fluxo de caixa no período $i$. O primeiro fluxo é o investimento inicial (negativo).
- $d_i$: É o nº de dias úteis desde o investimento inicial até o fluxo de caixa $i$.
- $P$:  **21 dias úteis** (período da taxa - mês comercial), resultando em uma TIR mensal.

### Definição das Variáveis usadas

Para cada variável categórica  (carteira total, um convênio ou um cedente- chamarei de "segmento"), as seguintes variáveis são calculadas a partir das parcelas a vencer:

* **$VP_{total}$**: O Valor Presente total do segmento, que representa o investimento inicial (saída de caixa).
* **$VN_i$**: O Valor Nominal da $i$-ésima parcela futura.
* **$PDD_{total}$**: A soma da Provisão para Devedores Duvidosos do segmento.
* **$C_i$**: O Custo Total (fixo + variável) associado à $i$-ésima parcela.
* **$RL_i$**: A Receita Líquida da $i$-ésima parcela, calculada como $RL_i = VN_i - C_i$.
* **$R_{PDD}$**: A taxa de PDD do segmento, calculada como $R_{PDD} = \frac{PDD_{total}}{VP_{total}}$.

Abaixo, a fórmula para cada uma das quatro TIRs calculadas.

### TIR Bruta a.m. (%)

Assume que todas as parcelas futuras serão recebidas em seu valor nominal completo, sem perdas por inadimplência ou custos operacionais (ideal).

* **Saída de Caixa (t=0):** $-VP_{total}$
* **Entradas de Caixa (t > 0):** $VN_i$

A equação resolvida é:
$$0 = -VP_{total} + \sum_{i=1}^{n} \frac{VN_i}{(1 + \text{TIR}_{\text{Bruta}})^{d_i/21}}$$

---

### TIR Líquida (PDD) a.m. (%)

Esta rentabilidade é ajustada pelo risco de crédito esperado. Ela mostra o retorno após considerar as perdas (projetadas) pela PDD. O investimento inicial não é alterado, mas o retorno futuro esperado é reduzido.

* **Saída de Caixa (t=0):** $-VP_{total}$
* **Entradas de Caixa (t > 0):** $VN_i \times (1 - R_{PDD})$

A equação resolvida é:
$$0 = -VP_{total} + \sum_{i=1}^{n} \frac{VN_i \times (1 - R_{PDD})}{(1 + \text{TIR}_{\text{PDD}})^{d_i/21}}$$

---

###  TIR Líquida (Custos) a.m. (%)

Esta rentabilidade é ajustada pelos custos operacionais. 

* **Saída de Caixa (t=0):** $-VP_{total}$
* **Entradas de Caixa (t > 0):** $RL_i$ (Receita Líquida)

A equação resolvida é:
$$0 = -VP_{total} + \sum_{i=1}^{n} \frac{RL_i}{(1 + \text{TIR}_{\text{Custos}})^{d_i/21}}$$

---

### TIR Líquida (PDD & Custos) a.m. (%)

Esta é a visão mais conservadora e completa, ajusta a rentabilidade tanto pelo risco de crédito quanto pelos custos operacionais.

* **Saída de Caixa (t=0):** $-VP_{total}$
* **Entradas de Caixa (t > 0):** $RL_i \times (1 - R_{PDD})$

A equação resolvida é:
$$0 = -VP_{total} + \sum_{i=1}^{n} \frac{RL_i \times (1 - R_{PDD})}{(1 + \text{TIR}_{\text{Completa}})^{d_i/21}}$$

---
---

## Mapeamento das Variáveis do Código para as Fórmulas

### DataFrames Principais

* `df_final2`: É o DataFrame principal que contém todos os dados da carteira após a limpeza inicial.
* `df_avencer`: É um subconjunto de `df_final2`, contendo apenas as parcelas com data de vencimento futura (`DataVencimento > ref_date`). É a base para todos os cálculos de TIR.
* `df_segmento`: Dentro do loop de análise, esta variável representa a "fatia" de `df_avencer` que corresponde ao segmento sendo analisado no momento (variável categ.).

### Variáveis de Cálculo (Dentro do Loop por Segmento)

| Variável no Código | Símbolo na Fórmula | Descrição e Cálculo |
| :--- | :--- | :--- |
| `vp_bruto` | $VP_{total}$ | O **Valor Presente** total do segmento, que representa o investimento inicial. É calculado como: `df_segmento['ValorPresente'].sum()`. |
| `pdd_total` | $PDD_{total}$ | A **Provisão para Devedores Duvidosos** total do segmento. Calculada como: `df_segmento['PDDTotal'].sum()`. |
| `pdd_rate` | $R_{PDD}$ | A **taxa de PDD** do segmento, que representa a perda esperada em relação ao valor presente. Calculada como: `pdd_total / vp_bruto`. |
| `df_avencer['_DIAS_UTEIS_']`| $d_i$ | Coluna com o número de **dias úteis** entre a data de referência e a data de vencimento de cada parcela `i`. |
| `df_avencer['CustoTotal']`| $C_i$ | Coluna com o **Custo Total** para cada parcela `i`, somando os custos fixos e variáveis. |
| `df_avencer['ReceitaLiquida']`| $RL_i$ | Coluna com a **Receita Líquida** para cada parcela `i`, calculada como `ValorNominal - CustoTotal`. |

### Construção do Fluxo de Caixa para a Função `calculate_xirr`

A função `calculate_xirr(cash_flows, days)` espera receber duas listas: uma com os valores e outra com os dias. O script as constrói da seguinte forma:

* **Fluxos de Caixa Futuros (Ex: `fluxos_brutos`)**:
    * É uma série `pandas` resultante de agrupar as parcelas por `_DIAS_UTEIS_` e somar os valores correspondentes (ex: `ValorNominal` para a TIR Bruta). O índice da série são os dias e os valores são os recebimentos totais naquele dia.
    * **Código:** `df_segmento.groupby('_DIAS_UTEIS_')['ValorNominal'].sum()`

* **Lista `cash_flows` (para a função):**
    * É A lista completa de fluxos de caixa.
    * **Código:** `[-vp_bruto] + fluxos_brutos.values.tolist()`
    * **Explicação:** O primeiro elemento é o investimento inicial **negativo** ($FC_0 = -VP_{total}$), seguido pelos recebimentos futuros.

* **Lista `days` (para a função):**
    * É A lista de tempo em dias úteis para cada fluxo de caixa.
    * **Código:** `[0] + fluxos_brutos.index.tolist()`
    * **Explicação:** O primeiro elemento é o dia 0 ($d_0 = 0$) para o investimento, seguido pelos dias úteis em que os recebimentos ocorrem.